# Projecte CIS-Congres

Després de fer el scraping de la web del Congrés i de obtenir els diaris de les sessions de control, procedim al seu anàl.lisi i a l'extracció dels diàlegs (documents: pregunta + intervencions).

# Analyzer

### Mètodes i utilitats

In [1]:
# http://stackoverflow.com/questions/7100125/storing-python-dictionaries

import json
from bson import json_util
import yaml

# http://api.mongodb.org/python/1.10.1/api/bson/json_util.html
# Mètodes per grabar y carregar fitxers en format json
def save_dict_json(dict, filename):
    with open(filename, 'wb') as fp:
        json.dump(dict, fp, default=json_util.default)

def load_dict_json(filename):
    with open(filename, 'rb') as fp:
        return json.load(fp, object_hook=json_util.object_hook)

# http://stackoverflow.com/questions/956867/how-to-get-string-objects-instead-of-unicode-ones-from-json-in-python
def load_dict_yaml(filename):
    with open(filename, 'rb') as fp:
        return yaml.load(fp)
    
# Mètode per grabar un fitxer de texte
def save_text_file(text, file_name, encoding = 'utf-8'):
    with open(file_name, "w") as text_file:
        if encoding:
            text_file.write(text.encode(encoding))
        else:
            text_file.write(text)

import ast
# Mètode per grabar un fitxer de texte
def save_list_text_file(text_list, file_name):
    print "Saving:", file_name, "lines:", len(text_list)
    with open(file_name, "w") as text_file:
        for item in text_list:
            text_file.write((r''.join(ast.literal_eval(item))+"\n").encode('utf-8'))
            # text_file.write(item+"\n")
        
# Mètode per carregar un fitxer de texte
def load_text_file(file_name, encoding = 'utf-8'):
    with open(file_name, "r") as text_file:
        if encoding:
            return text_file.read().decode(encoding)
        else:
            return text_file.read()
        
# Mètode per eliminar caracters incorrectes als noms de fitxers (potser millor utilitzar: https://pypi.python.org/pypi/goldfinch/0.4)
invalid_filename_chars_windows = '\/:*?"<>|'
def del_invalid_chars(value, deletechars):
    for c in deletechars:
        value = value.replace(c,'')
    return value;

import datetime

# Mètode per cambiar de format de data de format_1 a format_2
def format_date(strFecha, format_1, format_2):
    dtDate = datetime.datetime.strptime(strFecha, format_1)
    return dtDate.strftime(format_2)

# Mètode que ens diu si una data cau en dimecres
# https://docs.python.org/2/library/datetime.html#datetime.date.weekday
def isWednesday(my_date):
    if my_date.date().weekday() == 2:  # 0: Monday, 1: Tuesday, 2: Wednesday, ...
        return True
    else:
        return False
    
# print "24/03/2015", "isWednesday:", isWednesday("24/03/2015")
# print "11/03/2015", "isWednesday:", isWednesday("11/03/2015")

In [2]:
def get_text_between( s, first, last, include_limits = False, first_ocurrences = True, case_sensitive = True ):
    
    if not case_sensitive:
        ncs_s, ncs_first, ncs_last = s.lower(), first.lower(), last.lower()
    else:
        ncs_s, ncs_first, ncs_last = s, first, last
    
    try:
        if first_ocurrences:
            if include_limits:
                start = ncs_s.index( ncs_first )
                end = ncs_s.rindex( ncs_last, start ) + len(ncs_last)
            else:
                start = ncs_s.index( ncs_first ) + len( ncs_first )
                end = ncs_s.rindex( ncs_last, start ) 
        else: 
            # look for last ocurrences of first and last
            if include_limits:
                start = ncs_s.rindex( ncs_first )
                end = ncs_s.index( ncs_last ) + len(ncs_last)
            else:
                start = ncs_s.rindex( ncs_first ) + len( ncs_first )
                end = ncs_s.index( ncs_last ) 
            
        return s[start:end]            
    except ValueError:
        return ""
    
def remove_text_between( s, first, last, include_limits = False ):
    try:
        if include_limits:
            start = s.index( first ) + len( first )
            end = s.index( last, start ) 
            return s[:start] + s[end:] 
        else:
            start = s.index( first )
            end = s.index( last, start ) + len(last)
            return s[:start] + s[end:]
    except ValueError:
        return ""

import re

def remove_spaces_and_newline(s):
    return re.sub(r'\s+', ' ', s.replace('\n',''))


import unicodedata
import sys

# Mètode que treu els accents a un string (en format unicode)
def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

In [3]:
# de 0010_logging.ipynb
# http://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call
# http://stackoverflow.com/questions/28660700/stringio-getvalue-raising-unicodedecodeerror-when-printing-traceback

# https://docs.python.org/2/library/stringio.html#module-cStringIO
# cStringIO is faster than StringIO but:
# "Unlike the StringIO module, this module is not able to accept Unicode strings that cannot be encoded as plain ASCII strings.
import cStringIO
import traceback
import sys

class Capturing(list):
    def __enter__(self):
        # Save the stdout original value
        self._stdout = sys.stdout
        # Redirect the stdout to StringIO()
        sys.stdout = self._stringio = cStringIO.StringIO()
        return self
    def __exit__(self, *args):
        try:
            self.extend(self._stringio.getvalue().splitlines())
        except UnicodeDecodeError as e:
            print "[From Capturing]-------------------------------------------"
            print traceback.format_exc()
        finally:
            # Empty the buffer
            self._stringio.truncate(0)
            # Return to stdout original value
            sys.stdout = self._stdout

import ast
# Mètode per grabar un fitxer de texte
def save_list_text_file(text_list, file_name):
    print "Saving:", file_name, "lines:", len(text_list)
    with open(file_name, "w") as text_file:
        for item in text_list:
            text_file.write((r''.join(ast.literal_eval(item))+"\n").encode('utf-8'))
            # text_file.write(item+"\n")

def rprint(*obj):
    print u' '.join([repr('%s'%x) for x in obj])


In [4]:
# Reset to original sys.__stdout__
# sys.stdout = sys.__stdout__

#
with Capturing() as output:
    rprint(u"Hola...  pruebas y más pruebas")
    rprint(u'unicode')
    rprint(u'áéíóúÁÉÍÓÚñÑ')
    rprint(u"Adios.")

# Log output
print "Prueba"
print 'output:\n', output

save_list_text_file(output, "log/prueba.txt")

Prueba
output:
["u'Hola...  pruebas y m\\xe1s pruebas'", "u'unicode'", "u'\\xe1\\xe9\\xed\\xf3\\xfa\\xc1\\xc9\\xcd\\xd3\\xda\\xf1\\xd1'", "u'Adios.'"]
Saving: log/prueba.txt lines: 4


## Scraping de la web de documents del Congrés

### Base de dades del Congrés

#### Obrir conexió a la BD 'congres'

In [5]:
import pymongo

# Create the connection to MongoDB
try:
    connection=pymongo.MongoClient()
    print "Connection to Mongo Daemon successful!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e
    
# Obtenim la BD del Congrés
db = connection['congres']
#
print "Collections:", db.collection_names()

Connection to Mongo Daemon successful!!!
Collections: [u'system.indexes', u'all_document', u'congres_groups', u'congres_members', u'legislatures', u'document']


Check the database status
http://127.0.0.1:28017/

#### Tancar la conexió a la BD

In [39]:
connection.close()

#### Cerca de legislatures per data

In [6]:
import datetime

def get_legislature_DB(legislatures_col, d_doc_date):
    result = legislatures_col.find({'start':{'$lte':d_doc_date}, 'end':{'$gte':d_doc_date}})
    
    if result.count() == 1:
        return result[0]['_id']
    else:
        print "ERROR, get_legislature: date out of range:", doc_date
        return -1

# Test
legislatures_collection = db['legislatures']
    
my_date = datetime.datetime.strptime("03/06/2007", "%d/%m/%Y")
print "my_date:", my_date, ", legislature:", get_legislature_DB(legislatures_collection, my_date)
my_date = datetime.datetime.strptime("03/06/2012", "%d/%m/%Y")
print "my_date:", my_date, ", legislature:", get_legislature_DB(legislatures_collection, my_date)
my_date = datetime.datetime.strptime("03/06/1996", "%d/%m/%Y")
print "my_date:", my_date, ", legislature:", get_legislature_DB(legislatures_collection, my_date)
my_date = datetime.datetime.strptime("18/2/2015", "%d/%m/%Y")
print "my_date:", my_date, ", legislature:", get_legislature_DB(legislatures_collection, my_date)

my_date: 2007-06-03 00:00:00 , legislature: 8
my_date: 2012-06-03 00:00:00 , legislature: 10
my_date: 1996-06-03 00:00:00 , legislature: 6
my_date: 2015-02-18 00:00:00 , legislature: 10


#### Cerca de diputats a la BD

In [7]:
# Metode per buscar un diputat a la BD
# TODO: canviar la query per afegir el parametre 'legislature'
def search_diputado_in_BD(collection, s, n="", type='exact', legislature=0, verbose=False):
    if n=="":
        if type=='exact':
            result = collection.find({'surname': remove_accents(s.upper())})
        else:
            # search 'like'
            result = collection.find({'surname': {"$regex":remove_accents(s.upper())}})
    else:
        if type=='exact':
            result = collection.find({'surname': remove_accents(s.upper()), "name": remove_accents(n.upper())})
        else:
            # search 'like'
            result = collection.find({'surname': {"$regex":remove_accents(s.upper())}, "name": {"$regex":remove_accents(n.upper())}})
            
    if result.count() == 0:
        if verbose:
            print "WARN en search_diputado_in_BD_like: No s'han trobat resultats per:", s, "; ", n
        return None
    elif result.count() == 1:
        return result[0]
    else: # result.count() > 1:
        if verbose:
            print "WARN en search_diputado_in_BD_like: S'ha trobat més d'un resultat per: ", s, "; ", n
        if legislature > 0:
            f_result = []
            for result_obj in result:
                if verbose:
                    print "\t", result_obj
                found = False
                for group in result_obj['group']:
                    if legislature in result_obj['group'][group]:
                        found = True
                if found:
                    f_result.append(result_obj)

            if len(f_result) == 0:
                if verbose:
                    print "WARN en search_diputado_in_BD_like: No s'han trobat resultats per:", s, "; ", n, ", legislature:", legislature
                result.rewind()
                return result[0]
            elif len(f_result) == 1:
                return f_result[0]
            else:
                if verbose:
                    print "WARN en search_diputado_in_BD_like: S'ha trobat més d'un resultat per: ", s, "; ", n
                    for f_result_obj in f_result:
                        print "\t", f_result_obj
                return f_result[0]
        else:
            if verbose:
                print "WARN en search_diputado_in_BD_like: S'ha trobat més d'un resultat per: ", s, "; ", n
                for result_obj in result:
                    print "\t", result_obj
                # http://stackoverflow.com/questions/9872891/is-it-possible-to-iterate-a-mongo-cursor-twice
                result.rewind()
            return result[0]

In [8]:
def get_group(group_dict, legislature):
    for key in group_dict:
        if legislature in group_dict[key]:
            return key
    return group_dict.keys()[0]

''' '''
#
congres_members_collection = db['congres_members']

# El paràmetre de recerca en unicode
s = u"Llamazares Trigo"
n = u"Gaspar"
dip = search_diputado_in_BD(congres_members_collection,s,n)
if dip:
    print dip
    if len(dip['group']) == 1:
        print "Group:", dip['group'].keys()[0]
    else:
        print "Groups:", dip['group']
        group = get_group(dip['group'], 10)
        if group:
            print s, ",", n, "-->", group
else:
    print "ERROR en search_diputado_in_BD. No s'ha trobat:", s, n

print "-"*120
    
s = u"Fernández Díaz"
n = ""
dip = search_diputado_in_BD(congres_members_collection, s, legislature=10, verbose=True)
if dip:
    print dip
    if len(dip['group']) == 1:
        print "Group:", dip['group'].keys()[0]
    else:
        print "Groups:", dip['group']
        group = get_group(dip['group'], 10)
        if group:
            print s, ",", n, "-->", group
else:
    print "ERROR en search_diputado_in_BD. No s'ha trobat:", s

print "-"*120
    
s = u"DÍEZ GONZÁLEZ"
n = ""
dip = search_diputado_in_BD(congres_members_collection, s, legislature=10, verbose=True)
if dip:
    print dip
    if len(dip['group']) == 1:
        print "Group:", dip['group'].keys()[0]
    else:
        print "Groups:", dip['group']
        group = get_group(dip['group'], 10)
        if group:
            print s, ",", n, "-->", group
else:
    print "ERROR en search_diputado_in_BD. No s'ha trobat:", s

{u'group': {u'GIU': [7], u'GIP': [10], u'GER-IU-ICV': [9], u'GIU-ICV': [8]}, u'name': u'GASPAR', u'r_name': u'Gaspar', u'surname': u'LLAMAZARES TRIGO', u'_id': ObjectId('552594aa7c4c6704fc26710f'), u'r_surname': u'Llamazares Trigo'}
Groups: {u'GIU': [7], u'GIP': [10], u'GER-IU-ICV': [9], u'GIU-ICV': [8]}
Llamazares Trigo , Gaspar --> GIP
------------------------------------------------------------------------------------------------------------------------
WARN en search_diputado_in_BD_like: S'ha trobat més d'un resultat per:  Fernández Díaz ;  
	{u'group': {u'GP': [5]}, u'name': u'ELVIRA', u'r_name': u'Elvira', u'surname': u'FERNANDEZ DIAZ', u'_id': ObjectId('552594ab7c4c6704fc267576'), u'r_surname': u'Fern\xe1ndez D\xedaz'}
	{u'group': {u'GP': [5, 6, 7, 8, 9, 10]}, u'name': u'JORGE', u'r_name': u'Jorge', u'surname': u'FERNANDEZ DIAZ', u'_id': ObjectId('552594ab7c4c6704fc267577'), u'r_surname': u'Fern\xe1ndez D\xedaz'}
{u'group': {u'GP': [5, 6, 7, 8, 9, 10]}, u'name': u'JORGE', u'r_na

In [9]:
conjunctions = ['I', 'Y', 'DEL', 'DE']

def get_surname_name_from_string(speaker):
    speaker_list = speaker.upper().split()

    if len(speaker_list) > 1:
        if speaker_list[-2] in conjunctions:
            if len(speaker_list) > 4:
                surname = speaker_list[-3] + " " + speaker_list[-2] + " " + speaker_list[-1]
                name = speaker_list[0]
            elif len(speaker_list) == 4:
                surname = speaker_list[-3] + " " + speaker_list[-2] + " " + speaker_list[-1]
                name = speaker_list[-4]
            else:
                surname = speaker_list[-3] + " " + speaker_list[-2] + " " + speaker_list[-1]
                name = ""
        else:
            if len(speaker_list) > 2:
                if speaker_list[-3] == 'DE':
                    if len(speaker_list) > 4:
                        surname = speaker_list[-2] + " " + speaker_list[-1]
                        name = speaker_list[0]
                    else:
                        surname = speaker_list[-2] + " " + speaker_list[-1]
                        name = speaker_list[-4] + " DE"
                else:
                    if len(speaker_list) > 3:
                        surname = speaker_list[-2] + " " + speaker_list[-1]
                        name = speaker_list[0]
                    else:
                        surname = speaker_list[-2] + " " + speaker_list[-1]
                        name = speaker_list[-3]
            else: # len(speaker_list) == 2:
                surname = speaker_list[1]
                name = speaker_list[0]
    elif len(speaker_list) == 1:
        surname = speaker_list[0]
        name = ""
    else: # len(speaker_list) == 0:
        surname = ""
        name = ""

    return surname, name


def search_diputado_from_string(congres_members_col, speaker, legislature, verbose = False):
    surname, name = get_surname_name_from_string(speaker)
    return search_diputado_in_BD(congres_members_col, surname, name, type='like', legislature=legislature, verbose=verbose)


In [10]:
# Test del mètode search_diputado_from_string
#
congres_members_collection = db['congres_members']

for congres_member in congres_members_collection.find():
    ns = congres_member['r_name'] + " " + congres_member['r_surname']
    print congres_member['r_name'] + " | " + congres_member['r_surname'], "||", ns, get_surname_name_from_string(ns)
    dip = search_diputado_from_string(congres_members_collection, ns, legislature=10, verbose=True)
    if not dip:
        print "ERROR: no results."
        print "ns:", ns, "-->", congres_member['r_name'] + " || " + congres_member['r_surname']
        print surname, ";", name
        print "--------------------------------------------"

Luisa Fernanda | Rudi Ubeda || Luisa Fernanda Rudi Ubeda (u'RUDI UBEDA', u'LUISA')
Ramón | Rueda Espinar || Ramón Rueda Espinar (u'RUEDA ESPINAR', u'RAM\xd3N')
Juan Carlos | Ruiz López || Juan Carlos Ruiz López (u'RUIZ L\xd3PEZ', u'JUAN')
Francisco Javier | Rupérez Rubio || Francisco Javier Rupérez Rubio (u'RUP\xc9REZ RUBIO', u'FRANCISCO')
Antonio | Romero Ruiz || Antonio Romero Ruiz (u'ROMERO RUIZ', u'ANTONIO')
Victoriano | Roncero Rodríguez || Victoriano Roncero Rodríguez (u'RONCERO RODR\xcdGUEZ', u'VICTORIANO')
José Luis | Ros Maorad || José Luis Ros Maorad (u'ROS MAORAD', u'JOS\xc9')
María Amparo | Rubiales Torrejón || María Amparo Rubiales Torrejón (u'RUBIALES TORREJ\xd3N', u'MAR\xcdA')
Francesc Xavier | Sabaté Ibarz || Francesc Xavier Sabaté Ibarz (u'SABAT\xc9 IBARZ', u'FRANCESC')
Javier Luis | Sáenz Cosculluela || Javier Luis Sáenz Cosculluela (u'S\xc1ENZ COSCULLUELA', u'JAVIER')
Froilán Luis | Pérez González || Froilán Luis Pérez González (u'P\xc9REZ GONZ\xc1LEZ', u'FROIL\xc1N'

In [12]:
# Test 2: errors de transcripció. P.e. al doc 19980617_doc.html: "JERÓNIMO SAAVEDRAACEVEDO" hauria de ser "JERÓNIMO SAAVEDRA ACEVEDO"
#
congres_members_collection = db['congres_members']
legislatures_collection = db['legislatures']

my_date = datetime.datetime.strptime("17/06/1998", "%d/%m/%Y")
legislature = get_legislature_DB(legislatures_collection, my_date)
print search_diputado_from_string(congres_members_collection, u"JER\xd3NIMO SAAVEDRAACEVEDO", legislature, verbose = True)

my_date = datetime.datetime.strptime("20/12/2000", "%d/%m/%Y")
legislature = get_legislature_DB(legislatures_collection, my_date)
print search_diputado_from_string(congres_members_collection, u"MARIAASUNCIÓN OLTRA TORRES", legislature, verbose = True)

WARN en search_diputado_in_BD_like: No s'han trobat resultats per: SAAVEDRAACEVEDO ;  JERÓNIMO
None
WARN en search_diputado_in_BD_like: No s'han trobat resultats per: OLTRA TORRES ;  MARIAASUNCIÓN
None


La cerca de diputats no funciona amb errors de transcripció. Provarem amb una cerca fent servir 'String distance algorithms', per exemple, la distància de Levenshtein:

In [13]:
# http://es.wikipedia.org/wiki/Distancia_de_Levenshtein#Python
def distance_Levenshtein(str1, str2):
    n = len(str1)
    m = len(str2)
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    # return d[n][m]
    return 1.0 - float(d[n][m])/max(n,m)

# http://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# http://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Dice's_coefficient#Python
 
""" duplicate bigrams in a word should be counted distinctly
(per discussion), otherwise 'AA' and 'AAAA' would have a 
dice coefficient of 1...
"""
def dice_coefficient(a,b):
    if not len(a) or not len(b): return 0.0
    """ quick case for true duplicates """
    if a == b: return 1.0
    """ if a != b, and a or b are single chars, then they can't possibly match """
    if len(a) == 1 or len(b) == 1: return 0.0
 
    """ use python list comprehension, preferred over list.append() """
    a_bigram_list = [a[i:i+2] for i in range(len(a)-1)]
    b_bigram_list = [b[i:i+2] for i in range(len(b)-1)]
 
    a_bigram_list.sort()
    b_bigram_list.sort()
 
    # assignments to save function calls
    lena = len(a_bigram_list)
    lenb = len(b_bigram_list)
    # initialize match counters
    matches = i = j = 0
    while (i < lena and j < lenb):
        if a_bigram_list[i] == b_bigram_list[j]:
            matches += 2
            i += 1
            j += 1
        elif a_bigram_list[i] < b_bigram_list[j]:
            i += 1
        else:
            j += 1
 
    score = float(matches)/float(lena + lenb)
    return score

def search_diputado_in_BD_with_string_metric(collection, ns="", s="", n="", metric='distance_Levenshtein', verbose=False):
    
    dip_col = collection.find()
    
    cleaned_ns = ""
    if ns:
        cleaned_ns = remove_accents(ns.upper())
    elif n=="":
        cleaned_ns = remove_accents(s.upper())
    else:
        cleaned_ns = remove_accents(n.upper()) + ' ' + remove_accents(s.upper())
    
    result = [0,None]
    if cleaned_ns:
        if dip_col.count() > 0:
            for dip in dip_col:
                if ns or not n:
                    dip_ns = dip['name'] + ' ' + dip['surname']
                else:
                    dip_ns = dip['surname']

                if metric == 'dice_coefficient':
                    dist = dice_coefficient(cleaned_ns, dip_ns)
                else:
                    dist = distance_Levenshtein(cleaned_ns, dip_ns)

                if dist > result[0]:
                    result[0] = dist
                    result[1] = dip
                    # print result[0], ":", result[1]

    if verbose:
        print result[0], ":", result[1]

    return result[1]


In [14]:
# Test

#
congres_members_collection = db['congres_members']
legislatures_collection = db['legislatures']

my_date = datetime.datetime.strptime("17/06/1998", "%d/%m/%Y")
legislature = get_legislature_DB(legislatures_collection, my_date)

speaker = u"JER\xd3NIMO SAAVEDRAACEVEDO"
print remove_accents(speaker)
surname, name = get_surname_name_from_string(speaker)
print speaker, "-->", surname, ";", name
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, metric='dice_coefficient', verbose = True)
print "\n"
speaker = u"JOAN RIDAO 1 MARTÍN" # Trobat a 20101110_doc.html
surname, name = get_surname_name_from_string(speaker)
print speaker, "-->", surname, ";", name
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, metric='dice_coefficient', verbose = True)
print "\n"
speaker = u"MARIAASUNCIÓN OLTRA TORRES" # Trobat a 20001220_doc.html
surname, name = get_surname_name_from_string(speaker)
print speaker, "-->", surname, ";", name
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, metric='dice_coefficient', verbose = True)
print "----------------------------------------"
print search_diputado_in_BD_with_string_metric(congres_members_collection, s=surname, n=name, metric='dice_coefficient', verbose = True)


JERONIMO SAAVEDRAACEVEDO
JERÓNIMO SAAVEDRAACEVEDO --> SAAVEDRAACEVEDO ; JERÓNIMO
0.96 : {u'group': {u'GS': [6]}, u'name': u'JERONIMO', u'r_name': u'Jer\xf3nimo', u'surname': u'SAAVEDRA ACEVEDO', u'_id': ObjectId('552594ab7c4c6704fc2675dd'), u'r_surname': u'Saavedra Acevedo'}
{u'group': {u'GS': [6]}, u'name': u'JERONIMO', u'r_name': u'Jer\xf3nimo', u'surname': u'SAAVEDRA ACEVEDO', u'_id': ObjectId('552594ab7c4c6704fc2675dd'), u'r_surname': u'Saavedra Acevedo'}
----------------------------------------
0.6 : {u'group': {u'GS': [6]}, u'name': u'JERONIMO', u'r_name': u'Jer\xf3nimo', u'surname': u'SAAVEDRA ACEVEDO', u'_id': ObjectId('552594ab7c4c6704fc2675dd'), u'r_surname': u'Saavedra Acevedo'}
{u'group': {u'GS': [6]}, u'name': u'JERONIMO', u'r_name': u'Jer\xf3nimo', u'surname': u'SAAVEDRA ACEVEDO', u'_id': ObjectId('552594ab7c4c6704fc2675dd'), u'r_surname': u'Saavedra Acevedo'}
----------------------------------------
0.583333333333 : {u'group': {u'GS': [6]}, u'name': u'JERONIMO', u'r_name

In [15]:
speaker = u"MARIAASUNCIÓN OLTRA TORRES" # Trobat a 20001220_doc.html
surname, name = get_surname_name_from_string(speaker)
print speaker, "-->", surname, ";", name
print search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker)

MARIAASUNCIÓN OLTRA TORRES --> OLTRA TORRES ; MARIAASUNCIÓN
{u'group': {u'GP': [7, 8]}, u'name': u'MARIA ASUNCION', u'r_name': u'Mar\xeda Asunci\xf3n', u'surname': u'OLTRA TORRES', u'_id': ObjectId('552594ab7c4c6704fc26758b'), u'r_surname': u'Oltra Torres'}


#### Cerca de grups parlamentaris

In [18]:
def search_group_in_text_DB(text, congres_groups_col):
    
    for congres_group in congres_groups_col.find():
        description = congres_group['description'].upper()
        if description in text.upper():
            return congres_group['code']
    return None

def search_group_in_text_DB_v2(text, congres_groups_col):
    for congres_group in congres_groups_col.find():
        description = congres_group['description'].upper()
        if description in text.upper():
            return congres_group['code']
        
    if text.strip().upper() == 'POPULAR':
        return "GP"
    
    result = [0,None]
    for congres_group in congres_groups_col.find():
        dist = distance_Levenshtein(text.upper(), congres_group['description'].upper())
        if dist > result[0]:
            result[0] = dist
            result[1] = congres_group['code']
        
    return result[1]

# Test

doc_texts=[u'''
FEDERAL DE IZQUIEDA UNIDA
''',
u'''
 CATALÁN (CONVERGENCIA I UNIÓ)''',
u'''

    FEDERALDE IZQUIERDAUNIDA''',
u'''
    POPULAR EN ELCONGRESO''']

congres_groups_collection = db['congres_groups']

for doc_text in doc_texts:
    print "-"*120
    print "search_group_in_text_DB:", search_group_in_text_DB(remove_spaces_and_newline(doc_text), congres_groups_collection)
    print "search_group_in_text_DB_v2:", search_group_in_text_DB_v2(remove_spaces_and_newline(doc_text), congres_groups_collection)


------------------------------------------------------------------------------------------------------------------------
search_group_in_text_DB: None
search_group_in_text_DB_v2: GIU
------------------------------------------------------------------------------------------------------------------------
search_group_in_text_DB: None
search_group_in_text_DB_v2: GC-CiU
------------------------------------------------------------------------------------------------------------------------
search_group_in_text_DB: None
search_group_in_text_DB_v2: GIU
------------------------------------------------------------------------------------------------------------------------
search_group_in_text_DB: None
search_group_in_text_DB_v2: GP


### Estructura del diccionari de documents del Congrès

In [19]:
'''
all_document_dictionary = {
    'date': <data del document (Diario de sesiones del congreso de los diputados)>,
    'url': <url del document>,
    'description': <descripció del document>
    'id': <id del document a document_dictionary>
}

document_dictionary = {
    'date': <data del document (Diario de sesiones del congreso de los diputados)>,
    'url': <url del document>,
    'content_doc': <contingut del document en format html (sense tractar)>
    'updated_content_doc': <data d'actualització del camp 'content_doc'>
    <session_dictionary, lista amb les preguntes i intervencions de la sessió de control>
    'updated_session_dictionary': <data d'actualització del camp 'session_dictionary'>
}

session_dictionary = [{
    'question': <texte de la pregunta realitzada>,
    'speaker': <nom i cognoms de qui realitza la pregunta>,
    'group': <grup parlamentari que fa la pregunta o al que pertany qui realitza la pregunta>,
    'expedient_number': <número d'expedient de la pregunta>, 
    'intervention_dictionary' : [], llista amb les intervencions de la sessió de control i les persones que les van realitzar>
    }, ..., ]

intervention_dictonary = [{
    'text': <text de la intervenció>,
    'who': {  
        'surname': <cognoms de la persona que realitza la intervenció>,
        'name': <nom de la persona que realitza la intervenció>,
        'group': <grup al que pertany la persona: 'GP' (Grupo Popular), 'GS' (Grupo Socialista), ...>
    }, ..., ]
}
'''

"\nall_document_dictionary = {\n    'date': <data del document (Diario de sesiones del congreso de los diputados)>,\n    'url': <url del document>,\n    'description': <descripci\xc3\xb3 del document>\n    'id': <id del document a document_dictionary>\n}\n\ndocument_dictionary = {\n    'date': <data del document (Diario de sesiones del congreso de los diputados)>,\n    'url': <url del document>,\n    'content_doc': <contingut del document en format html (sense tractar)>\n    'updated_content_doc': <data d'actualitzaci\xc3\xb3 del camp 'content_doc'>\n    <session_dictionary, lista amb les preguntes i intervencions de la sessi\xc3\xb3 de control>\n    'updated_session_dictionary': <data d'actualitzaci\xc3\xb3 del camp 'session_dictionary'>\n}\n\nsession_dictionary = [{\n    'question': <texte de la pregunta realitzada>,\n    'speaker': <nom i cognoms de qui realitza la pregunta>,\n    'group': <grup parlamentari que fa la pregunta o al que pertany qui realitza la pregunta>,\n    'expedi

#### LListat dels documents seleccionats (sessions de control dels dimecres)

In [18]:
#
doc_col = db['document']

print "num. of rows: ", doc_col.count()
for doc in doc_col.find():
    file_name = doc['date'].strftime("%Y%m%d") + "_doc.html"
    print file_name, ":", doc['description']
    save_text_file(doc['content_doc'], 'docs/' + file_name)


num. of rows:  449
19980429_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 155, de 29/04/1998
19980520_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 161, de 20/05/1998
19971029_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 111, de 29/10/1997
19971008_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 106, de 08/10/1997
19970924_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 103, de 24/09/1997
19970507_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 81, de 07/05/1997
19970521_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 84, de 21/05/1997
19970528_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 87, de 28/05/1997
19970604_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 90, de 04/06/1997
19941130_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 111, de 30/11/1994
19941116_doc.html : Congreso de los Diputados, Pleno y Dip. Perm., núm. 104

In [16]:
'''
Blocs detectats en els diaris de sessions del Congrés dels dimecres (per ordre d'aparició):

"ORDEN DEL DÍA:"
- Trobat sempre

"SUMARIO"
- Trobat sempre


<br /><br />Se levanta la sesión a las ocho y cuarenta minutos de la noche.
<br /><br />Se suspende la sesión a las tres y diez minutos de la tarde.
<br /><br />Se reanuda la sesión a las nueve de la mañana.
    
    
"COMPARECENCIA DEL GOBIERNO ANTE EL PLENO DE LA CÁMARA"
- Trobat a DSCD-10-PL-172-C1.html

"PREGUNTAS"
- Trobat sempre

"INTERPELACIONES URGENTES:"
- Trobat sempre

"CONVALIDACIÓN O DEROGACIÓN DE REALES DECRETOS-LEYES:"
- Trobat a DSCD-10-PL-172-C1.html



'''

'\nBlocs detectats en els diaris de sessions del Congr\xc3\xa9s dels dimecres (per ordre d\'aparici\xc3\xb3):\n\n"ORDEN DEL D\xc3\x8dA:"\n- Trobat sempre\n\n"SUMARIO"\n- Trobat sempre\n\n\n<br /><br />Se levanta la sesi\xc3\xb3n a las ocho y cuarenta minutos de la noche.\n<br /><br />Se suspende la sesi\xc3\xb3n a las tres y diez minutos de la tarde.\n<br /><br />Se reanuda la sesi\xc3\xb3n a las nueve de la ma\xc3\xb1ana.\n    \n    \n"COMPARECENCIA DEL GOBIERNO ANTE EL PLENO DE LA C\xc3\x81MARA"\n- Trobat a DSCD-10-PL-172-C1.html\n\n"PREGUNTAS"\n- Trobat sempre\n\n"INTERPELACIONES URGENTES:"\n- Trobat sempre\n\n"CONVALIDACI\xc3\x93N O DEROGACI\xc3\x93N DE REALES DECRETOS-LEYES:"\n- Trobat a DSCD-10-PL-172-C1.html\n\n\n\n'

In [20]:
import re

def remove_html_tags(html_text):
    # Cleaning the text of html tags:
    # Remove:
    # <p style="text-align:center"><a name="(Página7)"><b>Página7</b></a></p>
    html_text = re.sub(r'<p.*?/p>', '', html_text, flags=re.DOTALL)
    
    # Remove:
    # <center><a name="(Página8259)"><b>Página 8259</b></a></center>
    html_text = re.sub(r'<center>.*?</center>', '', html_text, flags=re.DOTALL)
    
    # Remove:
    # <br />
    html_text = re.sub(r'<br />', '', html_text, flags=re.DOTALL)
    
    return html_text


log_to_file = True

def document_analyzer(document):

    legislature = get_legislature_DB(legislatures_collection, document['date'])
    print "date: ", document['date'], ", legislature:", legislature
    
    prefix_file_name = doc['date'].strftime("%Y%m%d") + "_doc"
    html_file = document['content_doc']
    
    #
    if u"EXCLUSIÓN DEL ORDEN DEL DÍA" in html_file:
        print "EXCLUSIÓN DEL ORDEN DEL DÍA: True"
        question_exclusion = True
    else:
        question_exclusion = False

    # Extraiem el text desde l'última aparició de 'PREGUNTAS' i '<hr size="1" noshade="" />' (aquest últim element només apareix al final de cada doc)
    preguntas_html = get_text_between(html_file, 'PREGUNTAS', '<hr size="1" noshade="" />', include_limits = False, first_ocurrences = False)

    preguntas_html = remove_html_tags(preguntas_html)

    # save_text_file(preguntas_html, 'log/' + prefix_file_name + '_preguntas_html.txt', "utf-8")
    
    paragraphs = preguntas_html.split("\n")
    
    #
    if log_to_file:
        text_to_analyze = ""
        i = 0
        for paragraph in paragraphs:
            text_to_analyze += "["+ str(i) + "]-----------------------------------------------------" + "\n"
            text_to_analyze += paragraph + "\n"
            i += 1
        # print text_to_analyze
        save_text_file(text_to_analyze, 'log/' + prefix_file_name + '_to_analyze.txt', "utf-8")

    #
    if log_to_file:
        with Capturing() as output:
            result_dialogue_dict = paragraphs_analyzer(paragraphs, preguntas_html, legislature, question_exclusion)
            # Log output
            # print 'output:\n', output
            
        #
        '''
        for item in output:
            print ast.literal_eval(item)
        '''
        save_list_text_file(output, 'log/' + prefix_file_name + '_analyzer_log.txt')

    else:
        result_dialogue_dict = paragraphs_analyzer(paragraphs, preguntas_html, legislature, question_exclusion)

    return result_dialogue_dict

In [21]:
def get_group_code(group_code):
    if group_code:
        for id in loaded_groups_dict:
            if loaded_groups_dict[id]['code'] == group_code:
                return loaded_groups_dict[id]['code']
    return ""

def get_who(surname, legislature):
    if surname:
        dip = search_diputado_in_BD(congres_members_collection, surname, legislature=legislature)
        if not dip:
            dip = search_diputado_in_BD_with_string_metric(congres_members_collection, surname)

        if dip:
            if len(dip['group']) == 1:
                dip_group = dip['group'].keys()[0]
            else:
                dip_group = get_group(dip['group'], legislature)

            who_dict = {'surname': dip['r_surname'], 'name': dip['r_name'], 'group': dip_group}
            return dip['_id'], who_dict

    who_dict = {'surname': '<unknown>', 'name': '<unknown>', 'group': '<unknown>'}
    return 9999, who_dict
    
def get_prev_non_empty_paragraph(paragraphs, from_index):
    i = from_index - 1
    while(i>=0):
        if paragraphs[i].split():
            return paragraphs[i]
        i+=-1
    return None

def get_next_non_empty_paragraph(paragraphs, from_index):
    i = from_index + 1
    while(i<len(paragraphs)):
        if paragraphs[i].split():
            return i,paragraphs[i]
        i+=1
    return 0, None
        
question_checkers = [u"DEL DIPUTADO", u"DE LA DIPUTADA", u"DEL GRUPO PARLAMENTARIO", u"DE DOÑA"]
# question_starters, és important l'ordre de les 
question_starters = [u"SOBRE", u"RELATIVA A", u"RELATIVA", u"ACERCA DE", u"PARA CONOCER SI", u"CLARIFIQUE",
                     u"EN MATERIA DE", u"DE CONOCER",u"RELACIÓN CON",u"PARA INFORMAR",u"EXPLIQUE",u"ACLARE",
                     u"INFORME",u"DEBATIR",u"EXPONGA",u"PARA",u":",u","]
    
def question_analyzer(sub_text, legislature):
    group = search_group_in_text_DB(remove_spaces_and_newline(sub_text), congres_groups_collection)

    # Remove the non-alphanumeric first characters
    m = re.search(r"\w", sub_text)
    if m:
        sub_text = sub_text[m.start():]

    if u'¿' in sub_text or u'?' in sub_text:
        question = get_text_between(sub_text,u'¿',u'?')
        if question == "":
            if u'¿' in sub_text:
                question = get_text_between(sub_text, u"¿", u"(Número", case_sensitive = False)
            elif u'?' in sub_text:
                # question = get_text_between(sub_text, u":", u"?")
                m = re.search('[\.|\:](.*?)\?', sub_text, re.DOTALL)
                if m:
                    question = sub_text[m.start():m.end()]

        if question == "":
            rprint("TO BE CATCHED -->", get_text_between(sub_text, u"GRUPO PARLAMENTARIO", u"(Número", case_sensitive = False))
            question = get_text_between(sub_text, u"GRUPO PARLAMENTARIO", u"(Número")                    

        if not group:
            group = get_text_between(sub_text, "GRUPO PARLAMENTARIO", ", QUE")

        # m = re.search(ur"(DIPUTADO SEÑOR|DIPUTADO DON|DIPUTADA DOÑA|SEÑORA DOÑA|DE DOÑA) ([^,]+),", sub_text)
        m = regex.search(ur"(DIPUTADO SEÑOR|DIPUTADO DON|DIPUTADO D.|SEÑOR DON|DIPUTADA DOÑA|SEÑORA DOÑA|DE DOÑA){e<=1} ([^,]+),", sub_text)
        if m:
            speaker = m.group(2)
        else:
            m = re.search(ur"(DIPUTADO|DON|DIPUTADA) ([^,]+),", sub_text)
            if m:
                speaker = m.group(2)
            else:
                catched_speaker = ""
                coma_index = sub_text.find(",")
                if coma_index > 0:
                    catched_speaker = sub_text[0:coma_index]
                else:
                    space_index = sub_text.find(" ", 40)
                    catched_speaker = sub_text[0:space_index]
                rprint("CATCHED -->", catched_speaker)
                
                s_words = question_checkers + [u"DIPUTADO", u"DIPUTADA", u"GRUPO", u"PARLAMENTARIO", u"DON", u"DOÑA",u"DE LA"]
                for s_word in s_words:
                    catched_speaker = catched_speaker.replace(s_word, " ")
                catched_speaker = remove_spaces_and_newline(catched_speaker)
                rprint("CATCHED (CLEANED) -->", catched_speaker)

                if not catched_speaker:
                    speaker = "not found"
                else:
                    speaker = catched_speaker
    else:
        for question_starter in question_starters:
            if question_starter in sub_text:
                question = get_text_between(sub_text, question_starter, u"(Número", case_sensitive = False)
                if not group:
                    group = get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", question_starter)
                break;

        if question=="":
            rprint("TO BE CATCHED -->", get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", u"(Número", case_sensitive = False))
            question = get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", u"(Número")

        speaker = "<group>"

    # GROUP
    group_code = get_group_code(group)
    if group_code:
        group = group_code
    else:
        group = search_group_in_text_DB_v2(remove_spaces_and_newline(group), congres_groups_collection)
    rprint("group:", group)

    # SPEAKER
    rprint("speaker:", speaker)
    if speaker!="<group>" and speaker!="not found":
        s_speaker = search_diputado_from_string(congres_members_collection, speaker, legislature=legislature)
        if not s_speaker:
            s_speaker = search_diputado_in_BD_with_string_metric(congres_members_collection, ns=speaker)

        # Obtenim el grup del speaker si no el tenim encara
        if s_speaker and not group:
            if len(s_speaker['group']) == 1:
                group = s_speaker['group'].keys()[0]
            else:
                group = get_group(dip['group'], legislature)
    else:
        s_speaker = speaker
    rprint("s_speaker:", s_speaker)
        
    return question, group, speaker, s_speaker


def intervention_analyzer(paragraphs, dialogue_dict, paragraph, fushioned, reparagraph, index, inc_index, who_id, p_offset, num_intervention, num_dialogue):
    m01 = re.search(ur"^\s*(El señor|La señora) (PRESIDENTE|PRESIDENTA)\:", paragraph)
    m02 = re.search(ur"^\s*(El señor|La señora) (VICEPRESIDENTE|VICEPRESIDENTA)\:", paragraph)
    m03 = re.search(ur"^\s*(El señor|La señora) (VICEPRESIDENTE|VICEPRESIDENTA) (\([^:]+\))\:", paragraph)
    m04 = re.search(ur"^\s*\(La señora vicepresidenta", paragraph)
    m05 = re.search(ur"^\s*\(El señor ([^:]+)\:", paragraph)
    if m01:
        rprint(u"KO (El señor|La señora) (PRESIDENTE|PRESIDENTA):", paragraph)
        p_offset += len(paragraph)
        who_id = None
    elif m02 or m03 or m04 or m05:
        rprint(u"KO (El señor|La señora) (VICEPRESIDENTE|VICEPRESIDENTA):", paragraph)
        p_offset += len(paragraph)
        who_id = None
    else:
        # https://pypi.python.org/pypi/regex [ regex 2015.03.18 ]
        
        # expressió regular per cognoms de dues paraules
        m1 = re.search(ur"señor\w? ([A-ZÁÉÍÓÚÑ\-]+) ([A-ZÁÉÍÓÚÑ\-]+)(\:|\.)", paragraph)
        if m1:
            rprint(u"OK 1", paragraph)
            rprint(u"found:", paragraph[m1.start():m1.end()])
            rprint(u"groups:", m1.group(0), ", ", m1.group(1), ", ", m1.group(2))
            rprint(u"speaker:", m1.group(1) + " " + m1.group(2))
            rprint(u"speach:", paragraph[m1.end():])

            who_id, who_dict = get_who(m1.group(1) + " " + m1.group(2), legislature)
                
            dialogue_dict[num_dialogue]['intervention_dictionary'].update({num_intervention:{'who':who_dict,'text':paragraph[m1.end():],'offset':p_offset}})
            p_offset = 0
            num_intervention += 1
            rprint("num_intervention:", num_intervention)
            if fushioned:
                index += inc_index
                rprint("index (f):", index, " inc:", inc_index)
        else:
            # expressió regular per els cognoms catalans
            m2 = re.search(ur"señor\w? ([A-ZÁÉÍÓÚÑ\-]+) I ([A-ZÁÉÍÓÚÑ\-]+)(\:|\.)", paragraph)
            if m2:
                rprint(u"OK 2", paragraph)
                rprint(u"found:", paragraph[m2.start():m2.end()])
                rprint(u"groups:", m2.group(0), ", ", m2.group(1), ", ", m2.group(2))
                rprint(u"speaker:", m2.group(1) + " I " + m2.group(2))
                rprint(u"speach:", paragraph[m2.end():])

                who_id, who_dict = get_who(m2.group(1) + " I " + m2.group(2), legislature)
                
                dialogue_dict[num_dialogue]['intervention_dictionary'].update({num_intervention:{'who':who_dict,'text':paragraph[m2.end():], 'offset':p_offset}})
                p_offset = 0
                num_intervention += 1
                rprint("num_intervention:", num_intervention)
                if fushioned:
                    index += inc_index
                    rprint("index (f):", index, " inc:", inc_index)
            else:
                # expressió regular per trobar intervencions del tipus: 
                #     "   El señor MINISTRO DE INDUSTRIA, ENERGÍA Y TURISMO (Soria López):"
                # m3 = re.search(ur"^\s*(El señor|La señora) ([^:]+)\:", paragraphs[index])
                m3 = regex.search(ur"^\s*(El señor|La señora){e<=1} ([^:]+)\:", paragraph)
                if m3:
                    rprint(u"OK 3", paragraph)
                    rprint(u"found:", paragraph[m3.start():m3.end()])
                    rprint(u"groups:", m3.group(0), ", ", m3.group(1), ", ", m3.group(2))
                    rprint(u"speaker:", m3.group(2))
                    if '(' in m3.group(2) and ')' in m3.group(2):
                        str_surname = remove_spaces_and_newline(get_text_between(m3.group(2), '(',')'))
                        # El señor LÓPEZ GARRIDO (don Diego):
                        if 'don' in str_surname:
                            str_surname = m3.group(2).split(' (')[0]
                    else:
                        str_surname = m3.group(2)
                    rprint(u"speaker2:", str_surname)
                    rprint(u"speach:", paragraph[m3.end():])

                    who_id, who_dict = get_who(str_surname, legislature)

                    dialogue_dict[num_dialogue]['intervention_dictionary'].update({num_intervention:{'who':who_dict,'text':paragraph[m3.end():], 'offset':p_offset}})
                    p_offset = 0
                    num_intervention += 1
                    rprint("num_intervention:", num_intervention)
                    if fushioned:
                        index += inc_index
                        rprint("index (f):", index, " inc:", inc_index)
                        
                else:
                    '''
                    m4 = regex.search(ur"(El señor|La señora) ([^:]+)\:", paragraph)
                    if m4 and not fushioned:
                        rprint(u"OK 4", paragraph)
                        rprint(u"found:", paragraph[m4.start():m4.end()])
                        rprint(u"groups:", m4.group(0), ", ", m4.group(1), ", ", m4.group(2))
                        rprint(u"speaker:", m4.group(2))
                        if '(' in m4.group(2) and ')' in m4.group(2):
                            str_surname = remove_spaces_and_newline(get_text_between(m4.group(2), '(',')'))
                            # El señor LÓPEZ GARRIDO (don Diego):
                            if 'don' in str_surname:
                                str_surname = m4.group(2).split(' (')[0]
                        else:
                            str_surname = m4.group(2)
                        rprint(u"speaker2:", str_surname)
                        rprint(u"speach:", paragraph[m4.end():])

                        who_id, who_dict = get_who(str_surname, legislature)

                        dialogue_dict[num_dialogue]['intervention_dictionary'].update({num_intervention:{'who':who_dict,'text':paragraph[m4.end():], 'offset':p_offset}})
                        p_offset = 0
                        num_intervention += 1
                        if fushioned:
                            index += inc_index
                    '''

                    index_mr = paragraph.find(u"El señor")
                    index_mrs = paragraph.find(u"La señora")
                    index_par = paragraph.find(u"(")
                    new_paragraph = ""
                    
                    if not reparagraph:
                        if index_mrs > 0 and index_par > 0:
                            new_paragraph = paragraph[index_mrs:index_par] + paragraph[0:index_mrs].lstrip() + paragraph[index_par:]
                        elif index_mr > 0 and index_par > 0:
                            new_paragraph = paragraph[index_mr:index_par] + paragraph[0:index_mr].lstrip() + paragraph[index_par:]
                    
                    if new_paragraph and not fushioned:
                        index, p_offset, num_intervention, who_id = intervention_analyzer(paragraphs, dialogue_dict, new_paragraph, fushioned, True, index, 0, who_id, p_offset, num_intervention, num_dialogue)

                    else:

                        if not fushioned:
                            next_index, next_non_empty_paragraph = get_next_non_empty_paragraph(paragraphs, index)
                            if next_non_empty_paragraph:
                                new_inc_index = next_index - index
                                paragraph_fusion = paragraphs[index] + next_non_empty_paragraph
                                rprint("Try paragraph_fusion (inc:", new_inc_index, ") -> ", paragraph_fusion)
                            else:
                                new_inc_index = 0
                                paragraph_fusion=""

                            index, p_offset, num_intervention, who_id = intervention_analyzer(paragraphs, dialogue_dict, paragraph_fusion, True, reparagraph, index, new_inc_index, who_id, p_offset, num_intervention, num_dialogue)

                        elif who_id:
                            rprint(u"OK ", who_id, ":")
                            # Eliminem la intervenció final del "PRESIDENTE" del congrés detectada en les intervencions de diaris de sessions antics
                            m = re.search(ur"(El señor|La señora) (PRESIDENTE|PRESIDENTA)\:", paragraphs[index])
                            if m:
                                extracted_text = paragraphs[index][0:m.start()]
                                rprint(u"(El señor|La señora) (PRESIDENTE|PRESIDENTA)\:")
                                rprint(u">" + extracted_text + u"<")
                            else:
                                extracted_text = paragraphs[index]
                                rprint(extracted_text)

                            dialogue_dict[num_dialogue]['intervention_dictionary'][(num_intervention-1)]['text'] += ("\n" + extracted_text)
                            rprint("Int:", (num_intervention-1), "->", dialogue_dict[num_dialogue]['intervention_dictionary'][(num_intervention-1)]['text'])
                        else:
                            rprint(u"KO ", paragraphs[index])
                            p_offset += len(paragraphs[index])
                        
    return index, p_offset, num_intervention, who_id


# El número d'expedient que té el següent format: 999/999999, amb excepcions:
                  # 20141015_doc.html -->  (Número de expediente 172/00263)
                  # 20101013_doc.htlm -->  (Número de expediente 180/0001223.)
                  # 20051214_doc.html -->  de expediente 180/ 001015.)
num_exp_regexs = [r"\D(\d{3}\s?/\s?\d{5,7})\D",
                  # 20030430_doc.html -->  (Número de expediente 180/1887.)
                  r"expediente (\d{3}\s?/\s?\d{4})\D",
                  # 20130417_doc.html -->  (Número de expediente 180000491)
                  # 20051019_doc.html -->  (Número de expediente 1807000927.)
                  # 20001220_doc.html -->  (Número de expediente\n1807000393.)
                  r"expediente\s*(\d{9,10})\D"]

def eval_regex_list(regex_list, paragraph):
    for regex_item in regex_list:
        m_q = re.search(regex_item, paragraph)
        if m_q:
            return m_q
    return m_q

# Extracció del número d'expedient
def get_num_exp(paragraph, paragraph_prev, num_exp_prev):

    m_q = eval_regex_list(num_exp_regexs, paragraph)
    if m_q:
        return m_q
    elif paragraph_prev:
        # 20000927_doc --> "INSTRUMENTOS DE CORRECCIÓN DE LOS MISMOS. (Número de expediente 172/\n000034.)
        p_paragraph = paragraph_prev.strip() + " " + paragraph.strip()
        # rprint("p_paragraph:",p_paragraph)
        m_q = eval_regex_list(num_exp_regexs, p_paragraph)
        if m_q and m_q.group(1) != num_exp_prev:
            return m_q
        else:
            return None
    return m_q

# List of words

ignored_sections = [u"INTERPELACIONES URGENTES:", u"CONVALIDACIÓN O DEROGACIÓN DE REALES DECRETOS-LEYES", u"JURAMENTO O PROMESA DE ACATAMIENTO DE LA CONSTITUCIÓN"]

# 20081029_doc --> "  retirada la pregunta. (Rumores.)"
# 20070509_doc --> "    decae esta interpelación"
# 20000628_doc --> " obviamente, se da por decaída. (Rumores.)"
remove_words = [u"aplazará",u"retirada",u"decae",u"Decae",u"decaída"]
        
        
import re
# https://pypi.python.org/pypi/regex
import regex
        
# Mètode que analitza una llista de paràgrafs
def paragraphs_analyzer(paragraphs, raw_text, legislature, question_exclusion):

    index = 0
    num_dialogue = 0
    dialogue_started = False
    num_intervention = 0
    dialogue_dict = {}
    who_id = None  # None: not interesting, 9999: unknown, otherwise: diputado id
    num_exp = ""
    sub_text = ""
    sub_text_started = False
    p_offset = 0
    
    continue_analyzing = True

    while index < len(paragraphs) and continue_analyzing:
        
        rprint("["+ str(index) + "]-----------------------------------------------------")
        
        # Extracció del número d'expedient
        m_q = get_num_exp(paragraphs[index],paragraphs[index-1],num_exp)
       
        if paragraphs[index].strip() == "":
            p_offset += len(paragraphs[index])
            rprint("Empty paragraph")
        elif not sub_text_started and any(question_checker in paragraphs[index] for question_checker in question_checkers):
            
            if dialogue_started:
                rprint("Dialog ends (1):", paragraphs[index])
                dialogue_started = False
                if num_intervention == 0:
                    prev_paragraph = get_text_between(raw_text, dialogue_dict[num_dialogue]['num_exp'], paragraphs[index])
                    if question_exclusion or any(remove_word in prev_paragraph for remove_word in remove_words):
                        rprint("Removing:",dialogue_dict[num_dialogue], "\nexclusion:", question_exclusion, "\nprev_paragraph:", prev_paragraph)
                        p_offset += len(dialogue_dict[num_dialogue]['question'])
                        dialogue_dict.pop(num_dialogue, None)
                        num_dialogue -= 1
                                
            rprint(u"sub_text_started = True")
            rprint(u"Adding:", paragraphs[index])
            sub_text = paragraphs[index] + "\n"
            sub_text_started = True
            
        elif m_q and sub_text_started:
            sub_text += paragraphs[index]
            rprint(u"sub_text: ", sub_text)
            sub_text_started = False
            
            num_dialogue += 1
            dialogue_started = True
            question = ""
            num_intervention = 0
            who_id = None

            num_exp = m_q.group(1)
            
            question, group, speaker, s_speaker = question_analyzer(sub_text, legislature)

            dialogue_dict[num_dialogue] = {'question':question, 'offset': p_offset, 'speaker':speaker, 'group':group, 'num_exp':num_exp, 'intervention_dictionary':{}}
            p_offset = 0
            rprint("num_exp:", num_exp)
            rprint("question:", question)
            rprint("s_speaker:", s_speaker)
        elif m_q and dialogue_started:
            rprint("Dialog ends (2):", paragraphs[index])
            dialogue_started = False
            if num_intervention == 0 and question_exclusion:
                rprint("Removing:",dialogue_dict[num_dialogue], "len dict:", len(dialogue_dict))
                p_offset += len(dialogue_dict[num_dialogue]['question'])
                dialogue_dict.pop(num_dialogue, None)
                rprint("Removed...  len dict:", len(dialogue_dict))
                num_dialogue -= 1
                
        elif sub_text_started:
            rprint(u"Adding:", paragraphs[index])
            sub_text += (paragraphs[index] + "\n")
            
        elif any(ignored_section in paragraphs[index] for ignored_section in ignored_sections):
            rprint(u"[DO NOTHING WITH] ignored_section:", paragraphs[index])
            dialogue_started = False
            if num_intervention == 0 and question_exclusion:
                rprint("Removing:",dialogue_dict[num_dialogue])
                p_offset += len(dialogue_dict[num_dialogue]['question'])
                dialogue_dict.pop(num_dialogue, None)
                num_dialogue -= 1
                
        elif u"PROPOSICIONES NO DE LEY" in paragraphs[index]:
            rprint(u"ANALYSIS END: 'PROPOSICIONES NO DE LEY", paragraphs[index])
            continue_analyzing = False
        elif u"MOCIONES CONSECUENCIA DE INTERPELACIONES URGENTES" in paragraphs[index]:
            rprint(u"ANALYSIS END: 'MOCIONES CONSECUENCIA DE INTERPELACIONES URGENTES':", paragraphs[index])
            continue_analyzing = False
        elif num_dialogue > 0 and dialogue_started:
            
            index, p_offset, num_intervention, who_id = intervention_analyzer(paragraphs, dialogue_dict, paragraphs[index], False, False, index, 0, who_id, p_offset, num_intervention, num_dialogue)
            
        else:
            rprint(u"[DO NOTHING] num_dialogue:", num_dialogue, u" >" + paragraphs[index] + u"<")

        index += 1
        rprint("index:", index, " inc: 1")
        
    if num_dialogue > 0 and len(dialogue_dict[num_dialogue]['intervention_dictionary'])==0:
        last_paragraph = get_prev_non_empty_paragraph(paragraphs,index)
        prev_paragraph = get_text_between(raw_text, dialogue_dict[num_dialogue]['num_exp'], last_paragraph)
        if question_exclusion or any(remove_word in prev_paragraph for remove_word in remove_words):
            rprint("Removing:",dialogue_dict[num_dialogue], "\nexclusion:", question_exclusion, "\nprev_paragraph:", prev_paragraph)
            p_offset += len(dialogue_dict[num_dialogue]['question'])
            dialogue_dict.pop(num_dialogue, None)

    return dialogue_dict


#### Procés d'anàl.lisi dels documents

In [24]:
import time

# Col.lecció de documents a la BD
doc_col = db['document']
# Col.lecció de diputats a la BD
congres_members_collection = db['congres_members']
# Col.lecció de legislatures a la BD
legislatures_collection = db['legislatures']
# Col.lecció de grups parlamentaris a la BD
congres_groups_collection = db['congres_groups']


# All
doc_start_date = "01/01/2000"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

'''
doc_date = "11/02/2015"
d_doc_date = datetime.datetime.strptime(doc_date, "%d/%m/%Y")
file_name = d_doc_date.strftime("%Y%m%d")
result = doc_col.find({'date': d_doc_date})
'''
'''
# Legislatura X
doc_start_date = "14/12/2011"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''
'''
# Legislaturas V-IX
doc_start_date = "29/06/1993"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "14/12/2011"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

print "Num. of docs:", result.count()
if result.count() > 0:
    
    t_total_init = time.clock()
    
    for doc in result:
        t_init = time.clock()
        print "Analyzing:", doc['description']
        result_dialogue_dict = document_analyzer(doc)
        # print result_dialogue_dict
        print "Updating '" + doc['description'] + "' ..."
        # save_dict_json(result_dialogue_dict, 'log/' + file_name + '_dialogue_dict.json')

        # dictionary conversion to session_dictionary
        session_dictionary = [None] * len(result_dialogue_dict)
        print "len(result_dialogue_dict):", len(result_dialogue_dict)
        for key in result_dialogue_dict:
            intervention_dictionary = [None] * len(result_dialogue_dict[key]['intervention_dictionary'])
            for subkey in result_dialogue_dict[key]['intervention_dictionary']:
                intervention_dictionary[int(subkey)] = result_dialogue_dict[key]['intervention_dictionary'][subkey]
            result_dialogue_dict[key]['intervention_dictionary'] = intervention_dictionary
            session_dictionary[int(key)-1] = result_dialogue_dict[key]

        # http://docs.mongodb.org/manual/tutorial/modify-documents/
        doc_col.update({'_id':doc['_id']}, \
                       {'$set':{'session_dictionary':session_dictionary, 'updated_session_dictionary':datetime.datetime.utcnow()}}, \
                       upsert=False, multi=False)
        t_end = time.clock()
        print "Done in {0:.2f} seconds".format(t_end-t_init)
        
    t_total_end = time.clock()
    print "\n" + ("-"*120)
    print "Analysis finished in {0:.2f} seconds".format(t_total_end-t_total_init)


Num. of docs: 317
Analyzing: Congreso de los Diputados, Pleno y Dip. Perm., núm. 7, de 17/05/2000
date:  2000-05-17 00:00:00 , legislature: 7
Saving: log/20000517_doc_analyzer_log.txt lines: 24252
Updating 'Congreso de los Diputados, Pleno y Dip. Perm., núm. 7, de 17/05/2000' ...
len(result_dialogue_dict): 26
Done in 15.51 seconds
Analyzing: Congreso de los Diputados, Pleno y Dip. Perm., núm. 9, de 24/05/2000
date:  2000-05-24 00:00:00 , legislature: 7
Saving: log/20000524_doc_analyzer_log.txt lines: 26183
Updating 'Congreso de los Diputados, Pleno y Dip. Perm., núm. 9, de 24/05/2000' ...
len(result_dialogue_dict): 26
Done in 11.20 seconds
Analyzing: Congreso de los Diputados, Pleno y Dip. Perm., núm. 11, de 31/05/2000
date:  2000-05-31 00:00:00 , legislature: 7
Saving: log/20000531_doc_analyzer_log.txt lines: 29482
Updating 'Congreso de los Diputados, Pleno y Dip. Perm., núm. 11, de 31/05/2000' ...
len(result_dialogue_dict): 27
Done in 15.32 seconds
Analyzing: Congreso de los Diputado

#### Mostra de l'últim diccionari processat

In [38]:
print result_dialogue_dict

{1: {'group': u'GMx', 'offset': 0, 'question': u'CONSIDERA EL GOBIERNO\n    ADECUADO PARA ERRADICAR LA POBREZA INFANTIL EN CATALU\xd1A LOS 2,25 EUROS\n    POR NI\xd1O QUE DESTIN\xd3 PARA COMBATIRLA MEDIANTE EL FONDO CORRESPONDIENTE', 'speaker': u'ALFRED BOSCH I PASCUAL', 'intervention_dictionary': [{'text': u' \xbfConsidera el Gobierno adecuados para erradicar\n    la pobreza infantil los 2,25 euros por ni\xf1o pobre que se dedican en\n    Catalu\xf1a mediante el fondo correspondiente?\n    Muchas gracias.', 'who': {'group': u'GMx', 'surname': u'Bosch i Pascual', 'name': u'Alfred'}, 'offset': 143}, {'text': u' Muchas gracias.\n    Si esa cifra fuera cierta, se\xf1or Bosch, me parecer\xeda absolutamente\n    inadecuada, por suerte es falsa, y yo puedo decir que a pesar de las\n    dificultades que hemos vivido en Catalu\xf1a y en el conjunto de Espa\xf1a a lo\n    largo de estos tiempos, somos un pa\xeds que hemos logrado que los espa\xf1oles\n    pudieran llevar una vida digna, en ocas

#### Paralelització de l'analisi [TODO]

In [44]:
from IPython import parallel
try:
    clients = parallel.Client()
    print clients.ids
except IOError as e:
    print e

[0, 1, 2, 3]


In [45]:
lview_all = clients.load_balanced_view()
print lview_all

<LoadBalancedView None>


In [47]:
def parallel_process_doc(doc):
    t_init = time.clock()
    print "Analyzing:", doc['description']
    result_dialogue_dict = document_analyzer(doc)

    print "Updating '" + doc['description'] + "' ..."
    # save_dict_json(result_dialogue_dict, 'log/' + file_name + '_dialogue_dict.json')

    # dictionary conversion to session_dictionary
    session_dictionary = [None] * len(result_dialogue_dict)
    print "len(result_dialogue_dict):", len(result_dialogue_dict)
    for key in result_dialogue_dict:
        intervention_dictionary = [None] * len(result_dialogue_dict[key]['intervention_dictionary'])
        for subkey in result_dialogue_dict[key]['intervention_dictionary']:
            intervention_dictionary[int(subkey)] = result_dialogue_dict[key]['intervention_dictionary'][subkey]
        result_dialogue_dict[key]['intervention_dictionary'] = intervention_dictionary
        session_dictionary[int(key)-1] = result_dialogue_dict[key]

    # http://docs.mongodb.org/manual/tutorial/modify-documents/
    doc_col.update({'_id':doc['_id']}, \
                   {'$set':{'session_dictionary':session_dictionary, 'updated_session_dictionary':datetime.datetime.utcnow()}}, \
                   upsert=False, multi=False)
    t_end = time.clock()
    print "Done in {0:.2f} seconds".format(t_end-t_init)

In [25]:
import time

# Col.lecció de documents a la BD
doc_col = db['document']
# Col.lecció de diputats a la BD
congres_members_collection = db['congres_members']
# Col.lecció de legislatures a la BD
legislatures_collection = db['legislatures']
# Col.lecció de grups parlamentaris a la BD
congres_groups_collection = db['congres_groups']

'''
# All
doc_start_date = "01/01/1990"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

doc_date = "19/10/2005"
d_doc_date = datetime.datetime.strptime(doc_date, "%d/%m/%Y")
file_name = d_doc_date.strftime("%Y%m%d")
result = doc_col.find({'date': d_doc_date})

'''
# Legislatura X
doc_start_date = "14/12/2011"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''
'''
# Legislaturas V-IX
doc_start_date = "29/06/1993"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "14/12/2011"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

if result.count() > 0:
    
    t_total_init = time.clock()
    
    async = lview_all.map(parallel_process_doc, result, block = False)
            
    t_total_end = time.clock()
    print "\n" + ("-"*120)
    print "Analysis finished in {0:.2f} seconds".format(t_total_end-t_total_init)


NameError: name 'lview_all' is not defined

#### Estadística de l'anàl.lisi dels resultats obtinguts

In [26]:
#
loaded_groups_dict = load_dict_json('groups_dict.json')

def get_group_name(group_code):
    if group_code:
        for id in loaded_groups_dict:
            if loaded_groups_dict[id]['code'] == group_code:
                return loaded_groups_dict[id]['name']
        print group_code, "not found"
    return ""

def search_group_in_text_v2(text, groups_dict):
    for key in groups_dict:
        description = groups_dict[key]['description'].upper()
        if description in text.upper():
            return groups_dict[key]['code']
        
    if text.strip().upper() == 'POPULAR':
        return "GP"
    
    result = [0,None]
    for key in groups_dict:
        dist = distance_Levenshtein(text.upper(), groups_dict[key]['description'].upper())
        if dist > result[0]:
            result[0] = dist
            result[1] = groups_dict[key]['code']
        
    return result[1]

In [27]:
# Col.lecció de documents a la BD
doc_col = db['document']
# Col.lecció de legislatures a la BD
legislatures_collection = db['legislatures']

# All
doc_start_date = "01/01/2000"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

'''
doc_date = "05/03/2003"
d_doc_date = datetime.datetime.strptime(doc_date, "%d/%m/%Y")
file_name = d_doc_date.strftime("%Y%m%d")
result = doc_col.find({'date': d_doc_date})
'''
'''
# Legislatura X
doc_start_date = "14/12/2011"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''
'''
# Legislaturas V-IX
doc_start_date = "29/06/1993"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "14/12/2011"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

if result.count() > 0 :
    print "Num. of docs:", result.count()
    for doc in result:
        print doc['description']
        legislature = get_legislature_DB(legislatures_collection, doc['date'])
        dialogue_stats = []
        for dialogue_id, dialogue in enumerate(doc['session_dictionary']):
            num_interventions = len(dialogue['intervention_dictionary'])
            dialogue_stats.append(num_interventions)
            if dialogue['question'].strip()=="":
                print "WARN: The question", dialogue['num_exp'] ,"is empty"
            if dialogue['question'].strip().count('\n')>10:
                print "WARN: The question", dialogue['num_exp'] ,"seems too long:", dialogue['question'].strip().count('\n'),"lines"
            if dialogue['speaker']=="not found":
                print "WARN: speaker of question", dialogue['num_exp'] ,"not found"
            if dialogue['group'].strip()=="":
                print "WARN: The group of question", dialogue['num_exp'] ,"is empty --> speaker:", dialogue['speaker']
                if dialogue['speaker']:
                    s_speaker = search_diputado_in_BD_with_string_metric(congres_members_collection, ns=dialogue['speaker'])
                    dip_group = ""
                    if s_speaker:
                        if len(s_speaker['group']) == 1:
                            dip_group = s_speaker['group'].keys()[0]
                        else:
                            dip_group = get_group(dip['group'], legislature)
                    print "--> dip_group:", dip_group
            else:
                group_name = get_group_name(dialogue['group'])
                if not group_name:
                    print "WARN: The group of question", dialogue['num_exp'] ,"not found"
                    print search_group_in_text_v2(remove_spaces_and_newline(dialogue['group']),loaded_groups_dict)
            if num_interventions < 2:
                print "WARN: The question", dialogue['num_exp'] ," has few interventions"
            else:
                for intervention_id, intervention in enumerate(dialogue['intervention_dictionary']):
                    if intervention['text'].strip()=="":
                        print "WARN: intervention", intervention_id, "of question", dialogue['num_exp'] ,"is empty"
                    if intervention['who']['surname']=='<unknown>':
                        print "WARN: who of intervention", intervention_id, "of question", dialogue['num_exp'] ,"is unknown"
                    if not intervention['who']['group']:
                        print "WARN: group of intervention", intervention_id, "of question", dialogue['num_exp'] ,"is empty -> who:", intervention['who']
                    else:
                        group_name = get_group_name(intervention['who']['group'])
                        if not group_name:
                            print "WARN: The group of intervention", intervention_id, "of question", dialogue['num_exp'] ,"not found"
                            print search_group_in_text_v2(remove_spaces_and_newline(intervention['who']['group']),loaded_groups_dict)
                
        print len(doc['session_dictionary']), ":", dialogue_stats


Num. of docs: 317
Congreso de los Diputados, Pleno y Dip. Perm., núm. 7, de 17/05/2000
26 : [4, 4, 2, 4, 3, 5, 4, 4, 4, 2, 2, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 2, 2, 7, 14, 11]
Congreso de los Diputados, Pleno y Dip. Perm., núm. 9, de 24/05/2000
26 : [2, 5, 3, 5, 4, 2, 2, 2, 4, 4, 4, 4, 3, 4, 4, 5, 2, 2, 2, 2, 2, 4, 4, 14, 18, 9]
Congreso de los Diputados, Pleno y Dip. Perm., núm. 11, de 31/05/2000
27 : [4, 4, 3, 3, 6, 9, 4, 4, 4, 4, 4, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 4, 13, 11, 8]
Congreso de los Diputados, Pleno y Dip. Perm., núm. 13, de 14/06/2000
25 : [4, 4, 2, 4, 4, 4, 4, 4, 6, 3, 2, 4, 2, 2, 2, 4, 2, 4, 2, 2, 4, 2, 2, 6, 8]
Congreso de los Diputados, Pleno y Dip. Perm., núm. 16, de 21/06/2000
27 : [4, 6, 2, 3, 4, 5, 4, 2, 2, 3, 4, 4, 4, 2, 2, 2, 5, 4, 4, 2, 2, 2, 2, 2, 12, 7, 9]
Congreso de los Diputados, Pleno y Dip. Perm., núm. 19, de 28/06/2000
26 : [4, 5, 6, 2, 2, 2, 4, 2, 5, 5, 2, 2, 4, 2, 2, 2, 2, 5, 2, 2, 4, 4, 4, 4, 6, 7]
Congreso de los Diputados, Pleno y Dip. Perm., núm

#### Marcatge dels paràgrafs seleccionats al document HTML

In [11]:
# constants de marcatge

# Afegint <a name="<MY_LABEL>"></a> a un determinat lloc a la pàgina, es pot accedir directament a aquesta part cridant href="#<MY_LABEL>"
form_tab_nav = u'<a name="{}"></a>'
form_tab_mark_start = u'<span name="dialog_mark" title="{}" style="background-color: {};">'
form_tab_mark_end = u'</span>'

def get_group_color_DB(congres_groups_col, group_code):
    
    result = congres_groups_col.find({'code':group_code})
    if result.count() > 0:
        return result[0]['color']
    else:
        print "WARN: group not found"
        return 'cyan'

def mark_text_in_HTML(html_page, title_text, selected_text, num_exp, start_index, group_color, verbose = False):
    
    tab_nav = ""
    if num_exp:
        if '/' in num_exp:
            tab_nav = form_tab_nav.format(num_exp.split('/')[1])
        else:
            tab_nav = form_tab_nav.format(num_exp)
    tab_mark_start = form_tab_mark_start.format(title_text, group_color)
    tab_mark_end = form_tab_mark_end
    
    text_start_index = html_page.find(selected_text, start_index)
    if text_start_index > 0:
        text_end_index = text_start_index + len(selected_text)
        if verbose:
            print "-"*20
            print "Searched:", selected_text
            print "Found:", selected_text[0:30]
            print "1. html_page len:", len(html_page)
            print "text_end_index:", text_end_index
        
        html_page = html_page[:text_end_index] + tab_mark_end + html_page[text_end_index:]
        html_page = html_page[:text_start_index] + tab_nav + tab_mark_start + html_page[text_start_index:]
        text_end_index += (len(tab_nav) + len(tab_mark_start) + len(tab_mark_end))
        
        if verbose:
            print "2. html_page len:", len(html_page), "t_inc:", (len(tab_mark_start) + len(tab_mark_end) + len(tab_nav))
            print "text_end_index:", text_end_index, ":", text_end_index - (len(tab_mark_start) + len(tab_mark_end) + len(tab_nav))
            print "->", html_page[text_end_index:text_end_index+80], "..."
    else:
        # print "S-->", selected_text
        # print "T-->", html_page[start_index:start_index+30]
        # Si la cerca del texte sencer no funciona, cerquem l'inici (fins '\n') i el final del texte
        first_break = selected_text.find("\n")
        last_break = selected_text.rfind("\n")
        if first_break > 0:
            if verbose:
                print first_break, ":", last_break
            text_start_index = html_page.find(selected_text[:first_break], start_index)
            text_end_index = html_page.find(selected_text[last_break:], start_index)
            if text_start_index > 0 and text_end_index > 0:
                text_end_index = text_end_index + len(selected_text[last_break:])
                html_page = html_page[:text_end_index] + tab_mark_end + html_page[text_end_index:]
                html_page = html_page[:text_start_index] + tab_nav + tab_mark_start + html_page[text_start_index:]
                text_end_index += (len(tab_nav) + len(tab_mark_start) + len(tab_mark_end))
            else:
                text_end_index = start_index
        else:
            text_end_index = start_index                

    return html_page, text_end_index

def get_marked_HTML_doc(selected_doc, use_offset = True, verbose = False):
    
    selected_content_doc = selected_doc['content_doc']

    text_index = selected_content_doc.rfind('PREGUNTAS')

    for dialogue_id, dialogue in enumerate(selected_doc['session_dictionary']):
        group_color = get_group_color_DB(congres_groups_collection, dialogue['group'])
        if use_offset:
            if verbose:
                print "dialogue['offset']:", dialogue['offset']
            text_index += dialogue['offset']
        question_text = dialogue['question'].strip()
        old_text_index = text_index
        if question_text=="":
            print "question: ", dialogue['num_exp'], "is empty."
        else:
            title_text = "dialogue: " + str(dialogue_id) + "\n" \
                + "group: " + dialogue['group'] + "\n" \
                + "speaker: " + dialogue['speaker'] + "\n" \
                + "num_exp: " + dialogue['num_exp']
            selected_content_doc, text_index = mark_text_in_HTML(selected_content_doc, title_text, question_text, dialogue['num_exp'], text_index, group_color, verbose = verbose)
            if text_index == old_text_index:
                print "text_type:", 'question', "text_index:", text_index, ", num_dialogue:", dialogue_id
                print ">" + question_text + "<"
        for inter_id, inter in enumerate(dialogue['intervention_dictionary']):
            paragraphs = inter['text'].split("\n")
            if verbose:
                print inter_id, "-> paragraphs:", paragraphs
            if use_offset:
                if verbose:
                    print "inter['offset']:", inter['offset']
                text_index += inter['offset']
            group_color = get_group_color_DB(congres_groups_collection, inter['who']['group'])

            for paragraph in paragraphs:
                old_text_index = text_index
                paragraph_text = paragraph.strip()
                if not paragraph_text=="":
                    title_text = "dialogue: " + str(dialogue_id) + "\n" \
                        + "num_inter: " + str(inter_id) + "\n"
                    if inter['who']['group']:
                        title_text += "group: " + inter['who']['group'] + "\n" \
                            + "who: " + inter['who']['surname'] + "; " + inter['who']['name']
                    selected_content_doc, text_index = mark_text_in_HTML(selected_content_doc, title_text, paragraph_text, "", text_index, group_color, verbose = verbose)
                    if text_index == old_text_index:
                        print "text_type:", 'intervention', "text_index:", text_index, ", num_dialogue:", dialogue_id, "num_inter:", inter_id
                        print ">" + paragraph_text + "<"

    return selected_content_doc

In [12]:
# Col.lecció de documents a la BD
doc_col = db['document']
# Col.lecció de diputats a la BD
congres_members_collection = db['congres_members']
# Col.lecció de legislatures a la BD
legislatures_collection = db['legislatures']
# Col.lecció de grups parlamentaris a la BD
congres_groups_collection = db['congres_groups']

'''
# All
doc_start_date = "01/01/2000"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

doc_date = "17/05/2000"
d_doc_date = datetime.datetime.strptime(doc_date, "%d/%m/%Y")
file_name = d_doc_date.strftime("%Y%m%d")
result = doc_col.find({'date': d_doc_date})

'''
# Legislatura X
doc_start_date = "14/12/2011"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''
'''
# Legislaturas V-IX
doc_start_date = "29/06/1993"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
doc_end_date = "14/12/2011"
d_doc_end_date = datetime.datetime.strptime(doc_end_date, "%d/%m/%Y")
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
'''

if result.count() > 0:
    for result_obj in result:

        file_name = result_obj['date'].strftime("%Y%m%d")
        selected_content_doc = get_marked_HTML_doc(result_obj, verbose = False)

        print "Saving 'docs/" + file_name + "_doc_marked.html'..."
        save_text_file(selected_content_doc, 'docs/marked/' + file_name + '_doc_marked.html',"utf-8")


Saving 'docs/20000517_doc_marked.html'...


#### Query documents, embedded documents

In [ ]:
# http://docs.mongodb.org/manual/reference/method/db.collection.update/
# http://docs.mongodb.org/manual/tutorial/query-documents/#read-operations-embedded-documents


#### Import and Export MongoDB Data

In [ ]:
# Import and Export MongoDB Data
# http://docs.mongodb.org/manual/core/import-export/

# http://stackoverflow.com/questions/13988479/loading-and-saving-mongodb-database-from-to-disk-with-pymongo

# Copying a Database
# http://api.mongodb.org/python/current/examples/copydb.html

# Export / Import
# http://stackoverflow.com/questions/11255630/how-to-export-all-collection-in-mongodb
# mongodump -d <our database name> -o <directory_backup>
# mongorestore <our database name>

## Utilitats per a les visualitzacions

#### Llistat de les dates dels documents seleccionats (sessions de control dels dimecres) començant desde setembre de 2000 (doncs és desde aquesta data on comencem a tenir baròmetres del CIS amb la pregunta que ens interessa "Tres problemas principales que existen actualmente en España (Multirrespuesta %)", com es pot veure a aquesta pàgina: http://www.cis.es/cis/export/sites/default/-Archivos/Indicadores/documentos_html/TresProblemas.html )

In [15]:
# Col.lecció de documents a la BD
doc_col = db['document']
# Col.lecció de diputats a la BD
congres_members_collection = db['congres_members']
# Col.lecció de legislatures a la BD
legislatures_collection = db['legislatures']
# Col.lecció de grups parlamentaris a la BD
congres_groups_collection = db['congres_groups']


In [16]:
import calendar
import datetime

def monthdelta(d1, d2):
    delta = 0
    while True:
        mdays = calendar.monthrange(d1.year, d1.month)[1]
        d1 += datetime.timedelta(days=mdays)
        if d1 <= d2:
            delta += 1
        else:
            break
    return delta

#
doc_col = db['document']
#
legislatures_col = db['legislatures']

# 
control_session_dates = []
months_without_control_session = []

doc_start_date = "01/09/2000"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

prev_date = d_doc_start_date
if result.count() > 0 :
    print "Num. of sessions:", result.count()
    for doc in result:
        control_session_dates.append("{}-{}-{}".format(doc['date'].day,doc['date'].month,doc['date'].year))
        # print doc['date'], ":", prev_date, " Legislatura:", get_legislature_DB(legislatures_col, doc['date'])
        print doc['date'], " Legislatura:", get_legislature_DB(legislatures_col, doc['date'])
        diff_months = monthdelta(prev_date,doc['date'])
        if diff_months > 0:
            for i in range(diff_months):
                elap_month = prev_date.month + 1 + i
                if elap_month > 12:
                    elap_month = elap_month - 12
                if elap_month!=doc['date'].month:
                    print "WARN: no sessions this month:", "{0}-{1}".format(elap_month, doc['date'].year)
                    months_without_control_session.append((elap_month,doc['date'].year))
        prev_date = doc['date']

print "-"*120
print "len(control_session_dates):", len(control_session_dates)
print "control_session_dates:", control_session_dates
print "-"*120
print "len(months_without_control_session):", len(months_without_control_session)
print "months_without_control_session:", months_without_control_session

Num. of sessions: 311
2000-09-13 00:00:00  Legislatura: 7
2000-09-20 00:00:00  Legislatura: 7
2000-09-27 00:00:00  Legislatura: 7
2000-10-04 00:00:00  Legislatura: 7
2000-10-18 00:00:00  Legislatura: 7
2000-11-29 00:00:00  Legislatura: 7
2000-12-13 00:00:00  Legislatura: 7
2000-12-20 00:00:00  Legislatura: 7
2001-02-07 00:00:00  Legislatura: 7
WARN: no sessions this month: 1-2001
2001-02-14 00:00:00  Legislatura: 7
2001-02-21 00:00:00  Legislatura: 7
2001-03-07 00:00:00  Legislatura: 7
2001-03-14 00:00:00  Legislatura: 7
2001-03-21 00:00:00  Legislatura: 7
2001-04-04 00:00:00  Legislatura: 7
2001-04-18 00:00:00  Legislatura: 7
2001-04-25 00:00:00  Legislatura: 7
2001-05-16 00:00:00  Legislatura: 7
2001-05-23 00:00:00  Legislatura: 7
2001-05-30 00:00:00  Legislatura: 7
2001-06-13 00:00:00  Legislatura: 7
2001-06-20 00:00:00  Legislatura: 7
2001-09-12 00:00:00  Legislatura: 7
WARN: no sessions this month: 7-2001
WARN: no sessions this month: 8-2001
2001-09-19 00:00:00  Legislatura: 7
200

#### Llistat de tuples de les dates de diaris de sessions de control amb el seu número de preguntes corresponent

In [17]:
#
doc_col = db['document']

control_session_dates = []

doc_start_date = "01/09/2000"
d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
d_doc_end_date = datetime.datetime.utcnow()
result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)

if result.count() > 0 :
    print "Num. of docs:", result.count()
    for doc in result:
        control_session_dates.append((doc['date'].strftime("%d-%m-%Y"),len(doc['session_dictionary'])))

print "control_session_dates:", control_session_dates


Num. of docs: 311
control_session_dates: [('13-09-2000', 24), ('20-09-2000', 27), ('27-09-2000', 25), ('04-10-2000', 22), ('18-10-2000', 28), ('29-11-2000', 24), ('13-12-2000', 28), ('20-12-2000', 22), ('07-02-2001', 27), ('14-02-2001', 26), ('21-02-2001', 27), ('07-03-2001', 25), ('14-03-2001', 27), ('21-03-2001', 25), ('04-04-2001', 28), ('18-04-2001', 26), ('25-04-2001', 27), ('16-05-2001', 24), ('23-05-2001', 27), ('30-05-2001', 26), ('13-06-2001', 25), ('20-06-2001', 28), ('12-09-2001', 22), ('19-09-2001', 30), ('26-09-2001', 24), ('03-10-2001', 26), ('31-10-2001', 26), ('28-11-2001', 31), ('12-12-2001', 27), ('19-12-2001', 27), ('06-02-2002', 27), ('13-02-2002', 24), ('20-02-2002', 29), ('06-03-2002', 26), ('13-03-2002', 25), ('20-03-2002', 26), ('10-04-2002', 27), ('17-04-2002', 26), ('24-04-2002', 26), ('08-05-2002', 23), ('22-05-2002', 27), ('29-05-2002', 11), ('12-06-2002', 27), ('19-06-2002', 25), ('26-06-2002', 27), ('11-09-2002', 27), ('18-09-2002', 24), ('25-09-2002', 16)

#### Llistat de grups parlamentaris amb els keywords que han utilitzat en un diari de sessions

In [18]:
import datetime

# Col.lecció de documents a la BD
doc_col = db['document']
#
# X LEGISLATURA  2011-Actualidad (Del 14 de diciembre de 2011 hasta la actualidad)
# doc_start_date = "14/12/2011"
# d_doc_start_date = datetime.datetime.strptime(doc_start_date, "%d/%m/%Y")
# d_doc_end_date = datetime.datetime.utcnow()
# result = doc_col.find({'date': {'$gte': d_doc_start_date, '$lte': d_doc_end_date}}).sort('date',1)
doc_date = "18/2/2015"
d_doc_date = datetime.datetime.strptime(doc_date, "%d/%m/%Y")
result = doc_col.find({'date': d_doc_date})

if result.count() > 0 :
    print "Num. of docs:", result.count()
    doc = result[0]
    print doc['description']

    legislature = get_legislature_DB(legislatures_collection, d_doc_date)
    print legislature
    
    group_keywords_dict = {}
    for dialogue_id, dialogue in enumerate(doc['session_dictionary']):
        for intervention_id, intervention in enumerate(dialogue['intervention_dictionary']):
            
            if intervention['who']['group']:
                s_group = intervention['who']['group']
            else:
                # print "intervention_id:", intervention_id, "who:", intervention['who'], "intervention:", intervention['text']
                print "intervention_id:", intervention_id, "who:", intervention['who']
                s_speaker = intervention['who']['name'] + " " + intervention['who']['surname']
                speaker = search_diputado_in_BD_with_string_metric(congres_members_collection, ns=s_speaker)
                # print speaker
                s_group = get_group(speaker['group'], legislature)
                
            if s_group in group_keywords_dict:
                keyword_list = [keyword[0] for keyword in intervention['keywords']]
                # print keyword_list
                group_keywords_dict[s_group]= list(set(group_keywords_dict[s_group] + keyword_list))
            else:
                group_keywords_dict[s_group]=[keyword[0] for keyword in intervention['keywords']]
            
    print len(group_keywords_dict), ":", group_keywords_dict.keys()
    result_list = "["
    for key in group_keywords_dict:
        result_list += ("['" + key + "', [" + ', '.join(("'" + keyword + "'") for keyword in group_keywords_dict[key]) + "]], ")
    result_list += "]"
    print result_list
else:
    print "No results."

Num. of docs: 1
Congreso de los Diputados, Pleno y Dip. Perm., núm. 260, de 18/02/2015
10
intervention_id: 0 who: {u'group': None, u'surname': u'D\xedez Gonz\xe1lez', u'name': u'Rosa Mar\xeda'}
intervention_id: 1 who: {u'group': None, u'surname': u'D\xedez Gonz\xe1lez', u'name': u'Rosa Mar\xeda'}
intervention_id: 3 who: {u'group': None, u'surname': u'D\xedez Gonz\xe1lez', u'name': u'Rosa Mar\xeda'}
intervention_id: 4 who: {u'group': None, u'surname': u'D\xedez Gonz\xe1lez', u'name': u'Rosa Mar\xeda'}
8 : [u'GIU', u'GV (EAJ-PNV)', u'GS', u'GIP', u'GP', u'GMx', u'GUPyD', u'GC-CiU']
[['GIU', ['operación huele', 'auténtica irresponsabilidad', 'registro civil', 'puesto fecha', 'comunidades autónomas', 'orden ministerial ilegal', 'servicio público esencial', 'tasas judiciales', 'justicia gratuita', 'operación', 'huele', 'auténtico escándalo', 'dirección contraria', 'sospechosa prisa', 'nacionalidad paralizados', 'operadores jurídicos']], ['GV (EAJ-PNV)', ['tontos perdóneme', 'cito literal', 

In [ ]:
import pickle

# Load
with open('km_20150419.pkl', 'rb') as input:
    km_loaded = pickle.load(input)

## Proves

In [47]:
num_exp_regexs = [r"\D(\d{3}/\d{6})\D",
                  # 20141015_doc.html -->  (Número de expediente 172/00263)
                  r"\D(\d{3}/\d{5})\D", 
                  # 20130417_doc.html -->  (Número de expediente 180000491)
                  r"\D(\d{9})\D"]

# Recerca del número d'expedient que té el següent format: 999/999999
def get_num_exp(paragraph):

    for num_exp_regex in num_exp_regexs:
        m_q = re.search(num_exp_regex, paragraph)
        if m_q:
            return m_q
    return m_q
    
paragraf = "    AL VIRUS DEL ÉBOLA. (Número de expediente 172/00263)."

print get_num_exp(paragraf).group(1)

172/00263


In [27]:
def question_analyzer(sub_text, legislature):
    group = search_group_in_text_DB(remove_spaces_and_newline(sub_text), congres_groups_collection)

    # Remove the non-alphanumeric first characters
    m = re.search(r"\w", sub_text)
    if m:
        sub_text = sub_text[m.start():]

    if u'¿' in sub_text or u'?' in sub_text:
        question = get_text_between(sub_text,u'¿',u'?')
        if question == "":
            if u'¿' in sub_text:
                question = get_text_between(sub_text, u"¿", u"(Número", case_sensitive = False)
            elif u'?' in sub_text:
                # question = get_text_between(sub_text, u":", u"?")
                m = re.search('[\.|\:](.*?)\?', sub_text, re.DOTALL)
                if m:
                    question = sub_text[m.start():m.end()]

        if question == "":
            rprint("TO BE CATCHED -->", get_text_between(sub_text, u"GRUPO PARLAMENTARIO", u"(Número", case_sensitive = False))
            # question = get_text_between(sub_text, u"GRUPO PARLAMENTARIO", u"(Número")                    

        if not group:
            group = get_text_between(sub_text, "GRUPO PARLAMENTARIO", ", QUE")
        m = re.search(ur"(DEL SEÑOR DIPUTADO DON|DEL DIPUTADO DON|DE LA DIPUTADA DOÑA|DE LA SEÑORA DOÑA) ([^,]+),", sub_text)
        if m:
            speaker = m.group(2)
        else:
            speaker = "not found"
    else:
        question_starters = [u"SOBRE", u"RELATIVA A", u"RELATIVA", u"ACERCA DE", u"PARA CONOCER SI", u"CLARIFIQUE",u"EN MATERIA DE", u"DE CONOCER",u"RELACIÓN CON",u"PARA INFORMAR",u"EXPLIQUE",u"ACLARE",u"INFORME",u"DEBATIR",u"EXPONGA",u"PARA",u":",u","]
        for question_starter in question_starters:
            if question_starter in sub_text:
                question = get_text_between(sub_text, question_starter, u"(Número", case_sensitive = False)
                if not group:
                    group = get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", question_starter)
                break;

        if question=="":
            rprint("TO BE CATCHED -->", get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", u"(Número", case_sensitive = False))
            # question = get_text_between(sub_text, u"DEL GRUPO PARLAMENTARIO", u"(Número")

        speaker = "<group>"

    rprint("group:", group)
    rprint("speaker:", speaker)
    if speaker!="<group>" and speaker!="not found":
        s_speaker = search_diputado_from_string(congres_members_collection, speaker, legislature=legislature, verbose = True)
    else:
        s_speaker = speaker
        
    return question, group, speaker, s_speaker

In [81]:
# import regex 

subtexts = [
u'''
    ? DE DIPUTADO DON ANTONIO CANTÓ GARCÍA DEL MORAL, DEL GRUPO PARLAMENTARIO
    DE UNIÓN PROGRESO Y DEMOCRACIA, QUE FORMULA AL SEÑOR MINISTRO DE ECONOMÍA
    Y COMPETITIVIDAD: ¿QUÉ MEDIDAS HA ADOPTADO EL GOBIERNO PARA GARANTIZAR
    QUE LOS ACCIONISTAS DE BANKIA ESTAFADOS PUEDAN RECUPERAR SU DINERO?
    (Número de expediente 180/001188).''',
u'''
    ? DE LA SEÑORA DOÑA ROSA MARÍA DÍEZ GONZÁLEZ, DEL GRUPO PARLAMENTARIO DE
    UNIÓN PROGRESO Y DEMOCRACIA, QUE FORMULA AL SEÑOR PRESIDENTE DEL
    GOBIERNO: ¿CREE USTED QUE SU GOBIERNO ESTÁ HACIENDO LO NECESARIO PARA
    GARANTIZAR LA IGUALDAD ENTRE ESPAÑOLES? (Número de expediente
    180/001204).''',
u'''
     - DEL DIPUTADO SEÑOR PÉREZ RUBALCABA, DEL GRUPO PARLAMENTARIO SOCIALISTA,
    QUE FORMULA AL SEÑOR PRESIDENTE DEL GOBIERNO: ¿CÓMO PIENSA AFRONTAR LA
    SITUACIÓN POLÍTICA DE ESPAÑA? (Número de expediente 180/000641).''',
u'''
     - DE DOÑA GEMA CONDE MARTÍNEZ, DEL GRUPO PARLAMENTARIO POPULAR EN EL
    CONGRESO, QUE FORMULA AL SEÑOR MINISTRO DEL INTERIOR: ¿QUÉ VALORACIÓN
    HACE EL MINISTERIO DEL INTERIOR DE LAS RECIENTES MODIFICACIONES LEGALES
    RELATIVAS A LA DESTRUCCIÓN DE DROGAS TÓXICAS Y ESTUPEFACIENTES
    APREHENDIDOS POR LAS FUERZAS Y CUERPOS DE SEGURIDAD DEL ESTADO? (Número
    de expediente 180/000590).''',
u'''
     - DEL DIPUTADO CON CARLOS MARTÍNEZ GORRIARÁN, DEL GRUPO PARLAMENTARIO DE
    UNIÓN PROGRESO Y DEMOCRACIA, QUE FORMULA AL SEÑOR MINISTRO DE HACIENDA Y
    ADMINISTRACIONES PÚBLICAS: ¿CÓMO EXPLICA EL GOBIERNO EL AUMENTO DE LA
    DEUDA DE LAS DIPUTACIONES PROVINCIALES EN UN 34 % DESDE 2008? (Número de
    expediente 180/000570).''',
u'''
     - EL DIPUTADO DON EDUARDO MADINA MUÑOZ, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA AL SEÑOR MINISTRO DE JUSTICIA: ¿POR QUÉ SE EMPEÑA
    EL GOBIERNO EN REFORMAR LA LEY DE INTERRUPCIÓN VOLUNTARIA DEL EMBARAZO?
    (Número de expediente 180/000558).''',
u'''
    - DE LA DIPUTADA, DOÑA ROSA DELIA BLANCO TERÁN, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA A LA SEÑORA MINISTRA DE SANIDAD, SERVICIOS
    SOCIALES E IGUALDAD: ¿QUÉ MEDIDAS VA A ADOPTAR EL GOBIERNO ANTE EL
    PROCESO QUE SE ESTÁ LLEVANDO A CABO EN LA SANIDAD DE LA COMUNIDAD DE
    MADRID? (Número de expediente 180/000547).''',
u'''
         - DEL DIPUTADO SABINO CUADRA LASARTE, DEL GRUPO PARLAMENTARIO MIXTO, QUE
    FORMULA AL SEÑOR MINISTRO DE DEFENSA: ¿HA CUMPLIMENTADO EL GOBIERNO LOS
    REQUERIMIENTOS DEL PARLAMENTO FORAL DE NAVARRA ADOPTADO MEDIANTE
    RESOLUCIÓN POR LA QUE SE INSTÓ A LO GOBIERNO A DEROGAR EL DECRETO EL
    DECRETO 411/1937, DE 8 DE NOVIEMBRE, POR EL QUE EL DICTADOR FRANCO
    CONCEDIÓ A NAVARRA LA CRUZ LAUREADA "COMO RECUERDO A LAS GESTAS HEROICAS
    DE NAVARRA EN EL MOVIMIENTO NACIONAL Y HOMENAJE A QUIEN RECIENTEMENTE
    ATESORA LAS VIRTUDES DE LA RAZA"? (Número de expediente 180/000483).''',
u'''
   - DE LA DIPUTADA, DOÑA ANA MARÍA ORAMAS GONZÁLEZ-MORO, DEL GRUPO
    PARLAMENTARIO MIXTO, QUE FORMULA AL SEÑOR PRESIDENTE DEL GOBIERNO:
    ¿CONSIDERA EL PRESIDENTE DEL GOBIERNO QUE HA CUMPLIDO LOS COMPROMISOS
    ASUMIDOS CON CANARIAS EN EL DEBATE DE INVESTIDURA? (Número de expediente
    180/000355).''',
u'''
       - DE LA SEÑORA DOÑA TERESA CUNILLERA I MESTRES, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA AL SEÑOR MINISTRO DE DEFENSA: ¿QUÉ VA A HACER EL
    GOBIERNO ANTE LOS PRONUNCIAMIENTOS DE UN MILITAR RETIRADO A FAVOR DE LA
    INTERVENCIÓN DE LAS FUERZAS ARMADAS EN COMUNIDADES AUTÓNOMAS? (Número de
    expediente 180/000239).''',
u'''
       - DE DON JOSÉ ANDRÉS AYALA SÁNCHEZ, DEL GRUPO PARLAMENTARIO POPULAR EN EL
    CONGRESO, QUE FORMULA A LA SEÑORA MINISTRA DE FOMENTO: ¿QUÉ OPORTUNIDADES
    SE HAN PODIDO IDENTIFICAR PARA LAS EMPRESAS ESPAÑOLAS DE INFRAESTRUCTURAS
    DURANTE SU VIAJE A BRASIL DE LA SEMANA PASADA? (Número de expediente
    180/000213).''',
u'''
      - DE LA DIPUTADA, DOÑA CARMEN MONTÓN GIMÉNEZ, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA A LA SEÑORA MINISTRA DE SANIDAD, SERVICIOS
    SOCIALES E IGUALDAD: ¿POR QUÉ LIMITA EL GOBIERNO LAS POLÍTICAS DE
    IGUALDAD? (Número de expediente 180/000065.)''',
u'''
        - DEL DIPUTADO, DON ALFREDO PÉREZ RUBALCABA, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA AL SEÑOR PRESIDENTE DEL GOBIERNO: ¿HA VALORADO EL
    GOBIERNO ADECUADAMENTE TODOS LOS EFECTOS DE SU REFORMA LABORAL? (Número
    de expediente 180/000040.)''',
u'''
       - DEL DIPUTADO JOSEP ANTONI DURAN I LLEIDA, DEL GRUPO PARLAMENTARIO
    CATALÁN (CONVERGÈNCIA I UNIÓ), QUE FORMULA AL SEÑOR PRESIDENTE DEL
    GOBIERNO: ¿CUÁL ES EL PORCENTAJE DE BECAS OTORGADAS POR EL ESTADO QUE HAN
    SIDO DESTINADAS A CATALUÑA EN LOS ÚLTIMOS CINCO AÑOS? (Número de
    expediente 180/001566.)''',
u'''
       - DE DOÑA MARÍA JESÚS BONILLA DOMÍNGUEZ, DEL GRUPO PARLAMENTARIO POPULAR
    EN EL CONGRESO, QUE FORMULA AL SEÑOR MINISTRO DE INDUSTRIA, TURISMO Y
    COMERCIO: ¿HA REALIZADO EL VICEPRESIDENTE TERCERO DEL GOBIERNO ALGUNA
    ACTUACIÓN DE COORDINACIÓN ENTRE ADMINISTRACIONES PARA ACLARAR LA
    INTERFERENCIA DE SEÑALES DE TELEVISIÓN PATROCINADA POR LA JUNTA DE
    COMUNIDADES DE CASTILLA-LA MANCHA? (Número de expediente 180/001426.)''',
u'''
   DEL DIPUTADO D. JULIO VILLARRUBIA MEDIAVILLA, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA AL SEÑOR MINISTRO DE JUSTICIA: ¿QUÉ ACTUACIONES
    ESTÁ LLEVANDO A CABO EL GOBIERNO PARA HACER REALIDAD LA MODERNIZACIÓN DEL
    SERVICIO  PÚBLICO DE LA JUSTICIA? (Número de expediente 180/000689.)''',
u'''
   - EL DIPUTADO DON JUAN CARLOS CORCUERA PLAZA, DEL GRUPO PARLAMENTARIO
    SOCIALISTA, QUE FORMULA AL SEÑOR MINISTRO DEL INTERIOR: ¿QUÉ VALORACIÓN
    HACE EL GOBIERNO DE LA APLICACIÓN DEL CÓDIGO PENAL EN MATERIA DE
    SEGURIDAD VIAL? (Número de expediente 180/000053.)''',
u'''    
       -DEL SEÑOR DON IÑAKI TXUEKA ISASTI, DEL GRUPO PARLAMENTARIO VASCO
    (EAJ-PNV), QUE FORMULA A LA SEÑORA MINISTRA DE AGRICULTURA, PESCA Y
    ALIMENTACIÓN: ¿CÓMO PLANTEA EL GOBIERNO AFRONTAR EL PROCESO DE
    REESTRUCTURACIÓN DEL SECTOR AZUCARERO ANTE LOS CAMBIOS QUE SE HAN IDO
    PRODUCIENDO DESDE QUE SE SUSCRIBIÓ EL ACUERDO MARCO INTERPROFESIONAL?
    (Número de expediente 180/001757.)''',
# 20100224_doc
u'''
       MARTÍNEZ SAIZ, DEL GRUPO PARLAMENTARIO POPULAR EN EL CONGRESO, QUE
    FORMULA AL SEÑOR MINISTRO DE FOMENTO: ¿CONSIDERA QUE LOS RECORTES
    PRESUPUESTARIOS EN EL MINISTERIO DE FOMENTO SE DEBEN TAMBIÉN A UNA
    CONSPIRACIÓN INTERNACIONAL? (Número de expediente 180/000869.)'''
]

for subtext in subtexts:
    print "-"*120
    # m = re.search(ur"(DIPUTADO SEÑOR|DIPUTADO DON|DIPUTADA DOÑA|SEÑORA DOÑA|DE DOÑA) ([^,]+),", subtext)
    m = regex.search(ur"(DIPUTADO SEÑOR|DIPUTADO DON|DIPUTADO D.|SEÑOR DON|DIPUTADA DOÑA|SEÑORA DOÑA|DE DOÑA){e<=1} ([^,]+),", subtext)
    if m:
        speaker = m.group(2)
    else:
        m = re.search(ur"(DIPUTADO|DON) ([^,]+),", subtext)
        if m:
            speaker = m.group(2)
        else:
            speaker = "not found"

    print speaker

    print question_analyzer(subtext, 10)

------------------------------------------------------------------------------------------------------------------------
ANTONIO CANTÓ GARCÍA DEL MORAL
'group:' u'GUPyD'
'speaker:' 'not found'
(u'QU\xc9 MEDIDAS HA ADOPTADO EL GOBIERNO PARA GARANTIZAR\n    QUE LOS ACCIONISTAS DE BANKIA ESTAFADOS PUEDAN RECUPERAR SU DINERO', u'GUPyD', 'not found', 'not found')
------------------------------------------------------------------------------------------------------------------------
ROSA MARÍA DÍEZ GONZÁLEZ
'group:' u'GUPyD'
'speaker:' u'ROSA MAR\xcdA D\xcdEZ GONZ\xc1LEZ'
(u'CREE USTED QUE SU GOBIERNO EST\xc1 HACIENDO LO NECESARIO PARA\n    GARANTIZAR LA IGUALDAD ENTRE ESPA\xd1OLES', u'GUPyD', u'ROSA MAR\xcdA D\xcdEZ GONZ\xc1LEZ', {u'group': {u'GUPyD': [10], u'GMx': [9]}, u'name': u'ROSA MARIA', u'r_name': u'Rosa Mar\xeda', u'surname': u'DIEZ GONZALEZ', u'_id': ObjectId('552594aa7c4c6704fc267417'), u'r_surname': u'D\xedez Gonz\xe1lez'})
-------------------------------------------------------

In [26]:
import re

text = '''
El señor PRESIDENTE DEL GOBIERNO (Aznar López): Sí, señor presidente.
'''
# paragraphs = text.split("\n\n")
# paragraphs = preguntas_html.split("\n\n")
# paragraphs = [text]

loaded_text_file = load_text_file('docs/19980429_doc.html',"")
preguntas_html = get_text_between(loaded_text_file, 'PREGUNTAS', '<hr size="1" noshade="" />')
paragraphs = preguntas_html.split("<br />")

# print remove_text_between(text, '(',')')

index = 0
while index < len(paragraphs):
    m = re.search(r"^\s*(El señor|La señora) (PRESIDENTE|PRESIDENTA|VICEPRESIDENTA)\:", paragraphs[index])
    if m:
        print "KO (El señor|La señora) (PRESIDENTE|PRESIDENTA):", paragraphs[index]
        who = None
    else:
        # expressió regular per cognoms de dues paraules
        m = re.search(r"señor\w? ([A-ZÁÉÍÓÚÑ\-]+) ([A-ZÁÉÍÓÚÑ\-]+)\:", paragraphs[index])
        if m:
            print "OK", paragraphs[index]
            print "found:", paragraphs[index][m.start():m.end()]
            print "groups:", m.group(0), ", ", m.group(1), ", ", m.group(2)
            print "speaker:", m.group(1) + " " + m.group(2)
            print "speach:", paragraphs[index][m.end():]

            id = search_diputado_in_dict(loaded_diputados_dict, m.group(1) + " " + m.group(2))
            # dip = search_diputado_in_BD(congres_members_collection, m.group(1) + " " + m.group(2))
            if id:
                who = id
            else:
                who = 9999
        else:
            # expressió regular per els cognoms catalans
            m = re.search(r"señor\w? ([A-ZÁÉÍÓÚÑ\-]+) I ([A-ZÁÉÍÓÚÑ\-]+)\:", paragraphs[index])
            if m:
                print "OK 2", paragraphs[index]
                print "found:", paragraphs[index][m.start():m.end()]
                print "groups:", m.group(0), ", ", m.group(1), ", ", m.group(2)
                print "speaker:", m.group(1) + " " + m.group(2)
                print "speach:", paragraphs[index][m.end():]

                id = search_diputado_in_dict(loaded_diputados_dict, m.group(1) + " I " + m.group(2))
                # dip = search_diputado_in_BD(congres_members_collection, m.group(1) + " I " + m.group(2))
                if id:
                    who = id
                else:
                    who = 9999           
            else:
                # expressió regular per trobar intervencions del tipus: 
                #     "   El señor MINISTRO DE INDUSTRIA, ENERGÍA Y TURISMO (Soria López):"
                m = re.search(r"^\s+(El señor|La señora) ([^:]+)\:", paragraphs[index])
                if m:
                    print "OK 3", paragraphs[index]
                    print "found:", paragraphs[index][m.start():m.end()]
                    print "groups:", m.group(0), ", ", m.group(1), ", ", m.group(2)
                    print "speaker:", m.group(2)
                    if '(' in m.group(2) and ')' in m.group(2):
                        str_surname = remove_spaces_and_newline(get_text_between(m.group(2), '(',')'))
                    else:
                        str_surname = m.group(2)
                    print "speaker2:", str_surname
                    print "speach:", paragraphs[index][m.end():]

                    id = search_diputado_in_dict(loaded_diputados_dict, str_surname)
                    # dip = search_diputado_in_BD(congres_members_collection, str_surname)
                    if id:
                        who = id
                    else:
                        who = 9999
    
    index += 1


KO (El señor|La señora) (PRESIDENTE|PRESIDENTA): El señor PRESIDENTE: Se reanuda la sesión.
OK El señor HERNANDEZ MOLTO: Muchas gracias.
found: señor HERNANDEZ MOLTO:
groups: señor HERNANDEZ MOLTO: ,  HERNANDEZ ,  MOLTO
speaker: HERNANDEZ MOLTO
speach:  Muchas gracias.
ERROR en search_diputado_in_dict. No s'ha trobat: HERNANDEZ MOLTO
OK El señor HERNANDEZ MOLTO: Muchas gracias, señor presidente.
found: señor HERNANDEZ MOLTO:
groups: señor HERNANDEZ MOLTO: ,  HERNANDEZ ,  MOLTO
speaker: HERNANDEZ MOLTO
speach:  Muchas gracias, señor presidente.
ERROR en search_diputado_in_dict. No s'ha trobat: HERNANDEZ MOLTO
KO (El señor|La señora) (PRESIDENTE|PRESIDENTA): El señor PRESIDENTE: Señora Rubiales, silencio.
KO (El señor|La señora) (PRESIDENTE|PRESIDENTA): El señor PRESIDENTE: Gracias, señor presidente del Gobierno.
KO (El señor|La señora) (PRESIDENTE|PRESIDENTA): El señor PRESIDENTE: Pregunta número 22, que formula al Gobierno el
    diputado don José Ignacio Echániz Salgado, del Grupo Pop

-c:17: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


#### Proves d'eliminació d'elements HTML amb expressions regulars

In [74]:
import re

text_html = '''
    <br /><br />El señor BOSCH I PASCUAL: ¿Considera el Gobierno adecuados para erradicar
    la pobreza infantil los 2,25 euros por niño pobre que se dedican en
    Cataluña mediante el fondo correspondiente?

    <br /><br />Muchas gracias.

    <br /><br /><br /><p style="text-align:center"><a name="(Página6)"><b>Página
    6</b></a></p><br />

    <br /><br />El señor PRESIDENTE: Muchas gracias, señor Bosch.

    <br /><br />Señor presidente.

    <br /><br />El señor PRESIDENTE DEL GOBIERNO (Rajoy Brey): Muchas gracias.
    
    
    
    <br /><br />El señor PRESIDENTE DEL GOBIERNO (Aznar López): Señor diputado, yo vengo
    aquí todos los miércoles y SS. SS. pueden preguntarme sobre cualquier
    aspecto de mi gestión política con toda tranquilidad,

    <br /><br /><center><a name="(Página8259)"><b>Página 8259</b></a></center>

    <br /><br />pero renunciar a ello y dedicarse a amparar o dar cobertura a
    afirmaciones injuriosas o caer directamente en afirmaciones que pueden    
'''

for m in re.finditer('ll', text):
    print('ll found', m.start(), m.end())


text_html = re.sub(r'<p.*?/p>', '', text_html, flags=re.DOTALL)
text_html = re.sub(r'<center>.*?</center>', '', text_html, flags=re.DOTALL)

print text_html


    <br /><br />El señor BOSCH I PASCUAL: ¿Considera el Gobierno adecuados para erradicar
    la pobreza infantil los 2,25 euros por niño pobre que se dedican en
    Cataluña mediante el fondo correspondiente?

    <br /><br />Muchas gracias.

    <br /><br /><br /><br />

    <br /><br />El señor PRESIDENTE: Muchas gracias, señor Bosch.

    <br /><br />Señor presidente.

    <br /><br />El señor PRESIDENTE DEL GOBIERNO (Rajoy Brey): Muchas gracias.
    
    
    
    <br /><br />El señor PRESIDENTE DEL GOBIERNO (Aznar López): Señor diputado, yo vengo
    aquí todos los miércoles y SS. SS. pueden preguntarme sobre cualquier
    aspecto de mi gestión política con toda tranquilidad,

    <br /><br />

    <br /><br />pero renunciar a ello y dedicarse a amparar o dar cobertura a
    afirmaciones injuriosas o caer directamente en afirmaciones que pueden    



In [60]:
string = '''
     --DEL DIPUTADO DON MANUEL JOSE SILVA I SANCHEZ, DEL GRUPO PARLAMENTARIO
    CATALAN (CONVERGéNCIA I UNIO), QUE FORMULA AL SEÑOR MINISTRO DE DEFENSA.
    CUAL ES EL GRADO DE UTILIZACION DEL CATALAN, LENGUA PROPIA Y OFICIAL DE
    CATALUÑA, EN LAS CAMPAÑAS INSTITUCIONALES DESARROLLADAS POR SU MINISTERIO
    A TRAVES DE LOS DIFERENTES MEDIOS DE COMUNICACION CON DIFUSION EN
    CATALUÑA? (Número de expediente 180/001192)
'''
m = re.search('[\.|\:](.*?)\?', string, re.DOTALL)
print m.start(),m.end(), ">", string[m.start():m.end()]

AttributeError: 'NoneType' object has no attribute 'start'

In [185]:
# https://pypi.python.org/pypi/regex

import regex

def get_speaker(paragraph):
    m3 = regex.search(ur"^\s*(El señor|La señora){e<=1} ([^{:.}]+)(\:|\.)", paragraph)
    if m3:
        print m3.fuzzy_counts
        print "(s substitutions, i insertions, d deletions)"
        print "OK 3", paragraph
        print "found:", paragraph[m3.start():m3.end()]
        print "groups:", m3.group(0), ", ", m3.group(1), ", ", m3.group(2)
        print "speaker:", m3.group(2)
        if '(' in m3.group(2) and ')' in m3.group(2):
            str_surname = remove_spaces_and_newline(get_text_between(m3.group(2), '(',')'))
        else:
            str_surname = m3.group(2)
        print "speaker2:", str_surname
        print "speach:", paragraph[m3.end():]

paragraphs = [u'     El seño BONO ARA: Gracias, señora presidenta.',
              u'     El señor BONO ARA: Gracias, señora presidenta.',
              u'     El señora BONO ARA: Gracias, señora presidenta.',
              u'    EL señor MINISTRO DE ASUNTOS EXTERIORES Y COOPERACIÓN (Moratinos Cuyaubé):',
              u'   (La señora vicepresidenta, Villalobos Talero, ocupa la Presidencia).',
              u'    El señor MINISTRO DE SANIDAD, SERVICIOS SOCIALES E IGUALDAD (Alonso     Aranegui)): Muchas gracias, señor presidente.',
              u'    El señor MINISTRO DE EDUCACIÓN, CULTURA Y DEPORTE (Wert Ortega): Muchas',
              u'   Y TECNOLOGÍA La señora MINISTRA DE CIENCIA (Birulés Bertrán):  ',
              u'   DE FOMENTO El señor MINISTRO (ÁlvarezCascos Fernández): Señora presidenta'
             ]

index = 0 
while index < len(paragraphs):
    print "----------------------------------------------------"
    '''
    m1 = regex.search(ur"(señor\w?){e<=3} ([A-ZÁÉÍÓÚÑ\-]+) ([A-ZÁÉÍÓÚÑ\-]+)\:", paragraphs[index])
    if m1:
        print m1.fuzzy_counts
        print "(s substitutions, i insertions, d deletions)"
        print u"OK 1", paragraphs[index]
        print u"found:", paragraphs[index][m1.start():m1.end()]
        print u"groups:", m1.group(0), ", ", m1.group(1), ", ", m1.group(2)
        print u"speaker:", m1.group(1) + " " + m1.group(2)
        print u"speach:", paragraphs[index][m1.end():]
    else:
        print "No results."
    '''
    m3 = regex.search(ur"^\s*(El señor|La señora){e<=1} ([^{:.}]+)(\:|\.)", paragraphs[index])
    if m3:
        print m3.fuzzy_counts
        print "(s substitutions, i insertions, d deletions)"
        print "OK 3", paragraphs[index]
        print "found:", paragraphs[index][m3.start():m3.end()]
        print "groups:", m3.group(0), ", ", m3.group(1), ", ", m3.group(2)
        print "speaker:", m3.group(2)
        if '(' in m3.group(2) and ')' in m3.group(2):
            str_surname = remove_spaces_and_newline(get_text_between(m3.group(2), '(',')'))
        else:
            str_surname = m3.group(2)
        print "speaker2:", str_surname
        print "speach:", paragraphs[index][m3.end():]
    else:
        print "No results."
        print "-"*120
        m4 = regex.search(ur"(El señor|La señora) ([^{:.}]+)(\:|\.)", paragraphs[index])
        if m4:
            print m4.fuzzy_counts
            print "(s substitutions, i insertions, d deletions)"
            print "OK 4", paragraphs[index]
            print "found:", paragraphs[index][m4.start():m4.end()]
            print "groups:", m4.group(0), ", ", m4.group(1), ", ", m4.group(2)
            print "speaker:", m4.group(2)
            if '(' in m4.group(2) and ')' in m4.group(2):
                str_surname = remove_spaces_and_newline(get_text_between(m4.group(2), '(',')'))
            else:
                str_surname = m4.group(2)
            print "speaker2:", str_surname
            print "speach:", paragraphs[index][m4.end():]
        print "-"*120
        index_mr = paragraphs[index].find(u"El señor")
        index_mrs = paragraphs[index].find(u"La señora")
        index_par = paragraphs[index].find(u"(")
        if index_mrs > 0:
            new_paragraph = paragraphs[index][index_mrs:index_par] + paragraphs[index][0:index_mrs].lstrip() + paragraphs[index][index_par:]
            print new_paragraph
        elif index_mr:
            new_paragraph = paragraphs[index][index_mr:index_par] + paragraphs[index][0:index_mr].lstrip() + paragraphs[index][index_par:]
            print new_paragraph
        get_speaker(new_paragraph)
        
    index += 1

----------------------------------------------------
(0, 0, 1)
(s substitutions, i insertions, d deletions)
OK 3      El seño BONO ARA: Gracias, señora presidenta.
found:      El seño BONO ARA:
groups:      El seño BONO ARA: ,  El seño ,  BONO ARA
speaker: BONO ARA
speaker2: BONO ARA
speach:  Gracias, señora presidenta.
----------------------------------------------------
(0, 0, 0)
(s substitutions, i insertions, d deletions)
OK 3      El señor BONO ARA: Gracias, señora presidenta.
found:      El señor BONO ARA:
groups:      El señor BONO ARA: ,  El señor ,  BONO ARA
speaker: BONO ARA
speaker2: BONO ARA
speach:  Gracias, señora presidenta.
----------------------------------------------------
(0, 1, 0)
(s substitutions, i insertions, d deletions)
OK 3      El señora BONO ARA: Gracias, señora presidenta.
found:      El señora BONO ARA:
groups:      El señora BONO ARA: ,  El señora ,  BONO ARA
speaker: BONO ARA
speaker2: BONO ARA
speach:  Gracias, señora presidenta.
----------------------